# <center>  NLP - HW2  </center>


In [ ]:
!pip install pint
!pip install hazm
!pip install unidecode
!pip install dadmatools

In [ ]:
import codecs
import tqdm
import re

In [ ]:
!git clone https://github.com/language-ml/parsi.io.git
%cd parsi.io

# add code below to __init__.py: (in parsi_io/modules/quantity_extractions/__init__.py)
'''
from .quantity_extractions import QuantityExtraction
from .unit_extractor import *
from .preprocessor import *
'''



Cloning into 'parsi.io'...
remote: Enumerating objects: 896, done.
remote: Counting objects: 100% (175/175), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 896 (delta 65), reused 61 (delta 14), pack-reused 721
Receiving objects: 100% (896/896), 52.41 MiB | 28.43 MiB/s, done.
Resolving deltas: 100% (296/296), done.
Checking out files: 100% (119/119), done.
/content/parsi.io/parsi.io


'\nfrom .quantity_extractions import QuantityExtraction\nfrom .unit_extractor import *\nfrom .preprocessor import *\n'

In [ ]:
from parsi_io.modules.quantity_extractions import *
from parsi_io.modules.number_extractor import NumberExtractor

### Load samples and units

<div dir=rtl>
در این بخش، تعدادی متن نمونه در حالت‌های مختلف قرار داده شده است تا پس از ایجاد مدل نهایی از آن برای تشخیص درستی مدل استفاده شود.

</div>

In [ ]:
samples = [
    'برنج سر لاشه گلچین امساله اعلاء ۱۰ کیلویی (تضمین کیفیت) هر کیسه با قیمت یک میلیون تومان به بازار عرضه می شود.',
    'قیمت روغن مخصوص سرخ کردنی بدون پالم اویلا ۱.۸ کیلوگرمی ، هر بطری ۲۰۰ هزار تومان می باشد.',
    'روغن حیوانی گاوی اعلاء نیم کیلویی با قیمت یکصد هزار تومان به دست مشتری می رسد.',
    'ماکارانی زر ماکارون قطر 1.5 وزن 700 گرم با قیمت ۱۵ هزار تومان برای هر بسته در فروشگاه های کشور توزیع می شود.',
    'نان حلوایی مجلسی کاسه ای بسته 60 عددی مناسب جهت انواع حلوا و سفره آرایی با قیمت ۸۰۰/۸ تومان در لوازم قنادی شادکام عرضه می شود.',
    'طبق اعلام ستاد موسوم به «تنظیم بازار کشور» قیمت هر کیلو مرغ ۵۹ هزار و ۸۰۰ تومان و هر کیلو تخم‌مرغ ۳۹ هزار و۸۰۰ تومان اعلام شد',
    'قیمت مصرف‌کننده هر کیلوگرم شیر خام ۳.۲ درصد چربی درب دامداری ۱۲۰,۰۰۰ ریال',
    'قیمت مصرف‌کننده شیر نایلونی ۹۰۰ گرمی کم چرب ۱۵۰,۰۰۰ ریال',
    'قیمت مصرف‌کننده شیر پاستوریزه بطری یک لیتری کم چرب ۱۸۰,۰۰۰ ریال',
    'قیمت مصرف‌کننده ماست دبه‌ای ۲/۵ کیلویی کم چرب ۴۹۸,۰۰۰ ریال',
    'قیمت مصرف‌کننده پنیر UF نسبتاً چرب ۴۰۰ گرمی ۳۷۰,۰۰۰ ریال',
    'قیمت مصرف کننده روغن مایع آفتاب گردان ۸۱۰ گرمی پخت و پز ۶۳۰,۰۰۰ ریال',
    'قیمت مصرف کننده روغن مایع ۸۱۰ گرمی سرخ کردنی ۶۲۰,۰۰۰ ریال',
    'قیمت ذرت خارجی ۲۰۳.۹ درصد نسبت به تیر سال قبل افزایش یافته و هر کیلوگرم آن از ۴۰۰۰ تومان به ۱۲ هزار و ۳۰۰ تومان رسیده است.', # When there is one product name with several prices
    'قیمت مصرف کننده هر کیلوگرم مرغ گرم ۵۹۸,۰۰۰ ریال',
    'قیمت مصرف کننده هر کیلوگرم تخم مرغ ۳۹۸,۰۰۰ ریال',
    'کیا ریو ۲۰۲۱ و ۲۰۲۲ جدیدترین مدل‌های نسل چهارم این خودرو هستند که با قیمت ۴۲۲ میلیون تومانی در کشورهای همسایه ایران خرید و فروش می‌شوند و جزو ارزان‌قیمت‌های بازارهای جهانی محسوب می‌شوند.',
    'قیمت کیا سورنتو 2019  اگر بخواهیم قیمت دلاری آن را محاسبه کنیم معادل حدود 77 هزار دلار است.',
    'قیمت استاندارد سراتو حدود 19 هزار و 400 دلار است',
    'قیمت جهانی سانتافه ۲۰۲۲ بین ۲۶ تا ۳۰ هزار دلار است که با دلار ۳۰ هزار تومانی در داخل کشور یعنی ۹۰۰ میلیون تومان می‌شود', # It takes the 30 dollars but doesn't take the 900 milions.
    'مگان ۲۰۲۲ اکنون با قیمت ۵۰۰ میلیون در امارات به فروش می رود',
    'نرخ یورو در بازار آزاد ۳۶۴,۷۲۰ ریال و نرخ پوند در بازار آزاد ۴۱۹,۷۸۰ ریال است.',
    'هزینه یک شب اقامت در اغلب هتل‌های مجهز (۵ ستاره) به ازای هر نفر بیشتر از ۲۰ دلار است.',
    'همچنین در سامرا مهمانسرای (مضیف) حضرت هر روز غذای رایگان توزیع می‌کند. ولی چنانچه قصد خرید داشته باشید، قیمت هر نان حدود ۲۵۰ دینار، استکان چای ۲۵۰ دینار می باشد.',
    'مبلغ تاکسی در دبی  برای هر کیلومتر در ترافیک روان 2 درهم است.',
    'هزینه جت اسکی به ازای هر نیم ساعت در دبی 125 درهم است.',
    'چاپ هر صفحه A3 در مجموعه های خدمات کامپیوتری برابر ۱۰۰۰ تومان می باشد.',
    'تحقیق موضوع فارسی هر صفحه کامل بدون پرینت با قیمت ۳۸۰۰۰ ریال در کافی نت ها انجام می شود.',
    'بسته به وضعیت سالمند و خدمات مورد نیاز سالمند، حقوق پایه ماهیانه پرستار روزانه معادل شش میلیون تومان می باشد.',
    'نظافت صفر تا صد منزل شرکت خدمات نظافتی اصفهان پاک، آماده پذیرش درخواست های شما در خصوص نظافت منزل با قیمت هر ساعت ۱۰۰ هزار تومان می باشد.',
    'وتیروکسین که با ۳۴ درصد کمترین افزایش نرخ را داشت و به ۳۵ هزار تومان افزایش یافت.',
    'هر ورق قرص اورسودزوکسی کولیک به یکباره به ۵۱ هزار تومان رسید.',
    'هر گرم طلا ۱۸ عیار در بازار امروز نسبت به روز قبل ۱,۴۹۴,۴۰۰ تومان نرخ گذاری شد.',
    'هر مثقال طلا 18 عیار در بازار امروز ۶,۴۷۳,۰۰۰ تومان نرخ گذاری شد.',
    'هر اونس طلا در بازار امروز نسبت به ۲ روز پیش ، ۱,۷۴۵ نرخ گذاری شد.',
    'بر این اساس قیمت هر کیلوگرم جو و ذرت ۱۱۳۰۰ تومان و کنجاله سویا ۱۶۳۰۰ تومان تعیین شد',
    'به گزارش پایگاه خبری اقتصاد غذا «فود پرس»، نرخ جو فرانسه هر تن ۲۵۷ یورو می باشد.',
    'در   بازار گل محلاتی تهران هر شاخه گل نرگس به قیمت ۵۰,۰۰۰ ریال فروخته می شود.',
    'بسته های فرهنگی هر کدام به ارزش ۲۰ هزار تومان بین افراد حاضر در جلسه توزیع شد.',
    'ماشین های ۲ میلیارد تومانی، برای پاداش مدال آوران المپیک در نظر گرفته شد.',
    'تلویزیون ال ای دی هوشمند سام سایز ۵۸ اینچ هر دستگاه به قیمت ۱۰۰ میلیون تومان فروخته می شود.',
    'پمپ تخلیه لباسشویی ۱۰ کیلویی سامسونگ در نمایندگی این شرکت به قیمت   ۱٫۲۵۰٫۰۰۰ تومان  فروخته می شود.',
    'کرایه باربری تهران به مشهد  برای کامیون تک ،‌ معادل ۱۰۰ تومان به ازای هر کیلومتر می باشد.',
    'قیمتی که توسط شورای قیمت گذاری محصولات کشاورزی برای گندم ۱۴۰۱-۱۴۰۲ اعلام شده، ۱۳ هزار تومان است',
    'در حال حاضر هر تن گندم در بازار جهانی ۳۳۱ یورو قیمت‌گذاری شده است.',
    'یک کیلوگرم گندم حدودا معادل ۱۰۱۶۰ تومان می‌شود.',
    'در جهان حدود ۲ میلیارد تن زباله شهری و صنعتی تولید می‌شود که ارزش بازار آن به ۵۰۰ میلیارد دلار می‌‌رسد.',
    'به گزارش واحد خبرنگار اقتصادی خبرگزاری تسنیم، هم اکنون در بازار آزاد، قیمت طلای۱۸ عیار هر گرم یک میلیون و ۳۵۹ هزار تومان و قیمت سکه تمام بهار آزادی طرح جدید ۱۵ میلیون و ۱۰۰ هزار تومان است',
    'دبیر انجمن تصفیه کنندگان شکر گفت: در شرایط کنونی هر تن شکر ۵۳۰ دلار تحویل بنادر می شود.',
    'در شرکت اسپارد،‌ قیمت یک سرویس نظافت ۵ ساعته، ۲۴ هزار تومان می باشد.',
    'مبادلات تجاری کشور در هفت ماه ابتدایی سال جاری حدود ۸۱ میلیون تن به ارزش ۶۰.۱ میلیارد دلار بوده است',
    'به گزارش ایسنا، براساس مشاهدات میدانی از خرده‌فروشی‌ها سطح شهر تهران قیمت هر کیلوگرم پیاز ۱۶ تا ۲۲ هزار تومان، گوجه فرنگی حدود ۱۶ هزار تومان، خیار ۱۷ هزار و ۵۰۰ تومان، سیب زمینی ۱۵ هزار تومان، موز ۴۳ تا ۵۳ هزار تومان، سیب قرمز ۴۷ هزار تومان، پرتقال ۱۸ هزار و ۹۰۰ تومان، نارنگی ۱۷ تا ۳۳ هزار تومان و گلابی ۳۱ هزار تومان است.',
    'قیمت مرغ پس از اصلاح ارز ترجیحی به 59 هزار و 500 تومان و پس از آن با انتقاد تولید‌کنندگان کیلویی 63 هزار تومان مصوب شد اما بررسی میدانی نشان داد که قیمت مرغ در منطقه 19 تهران کیلویی 52 هزار تومان است.',
    'به گزارش شبکه شرق، قیمت کی ام سی T۸ مدل ۱۴۰۱ در هفته گذشته، به یک میلیارد و ۱۹۵ میلیون تومان رسیده‌است.',
    'قیمت یک بطری ۳۵۰ میلی‌لیتری کوکاکولا برای خارجی‌هایی که به قصد تماشای بازی‌های جام‌جهانی عازم قطر شده‌اند، در هتل‌های کشور میزبان، ۱۵ ریال است. هر دلار آمریکا با ۳.۶ ریال قطر برابری می‌کند.',
    'قیمت نوشابه کوکاکولا در بطری‌های فلزی قرمزرنگ، با حجم ۳۳۰ میلی‌لیتر، حدود ۹ هزار و ۷۰۰ تومان تا ۱۲ هزار تومان است.',
    'قیمت یک پیاله کوچک بادام زمینی نمکی، معادل ۲۹۴ هزار تومان می‌رسد و برای همان مقدار پسته باید معادل ۳۴۳ هزار تومان پول پرداخت کرد.',
    'کاهش قیمت ها در برابر قیمت زمین متری ۱۵۰ میلیون تومان در شمال تهران و متری ۳۰ میلیون تومان در جنوب تهران رقمی نیست.',
    'با توجه به اينكه مسكو قصد دارد از پنجم دسامبر بخشي از نفت را با نرخ 60 دلار به بازار آسيا تزريق كند، بسياري از كارشناسان به دولت ايران پيشنهاد داده‌اند كه در لايحه بودجه قيمت نفت را بين 40 تا 50 دلار و با سقف صادراتي 600 هزار بشكه در روز در نظر بگيرد.',
    'ارزیابی بازار نشان دهنده آن است که خانواده کوییک هم بدون تغییر راهی بازار شدند.  با این شرایط  «کوییکR » در بازار به  229 میلیون تومانی قیمت خورد و «کوییک معمولی» هم در بازار 222 میلیون تومانی شد.',
    'معاملات توافقی دلار در بازار ارز امروز شنبه ۵ آذر۱۴۰۱ با قیمت خرید ۳۳هزار و ۵۰۰ تومان کار خود را آغاز کرد.',
    'طی امروز قیمت هر کیلوگرم پیاز قرمز بالای ۱۵ هزار تومان، پیاز شیری ۱۲ هزار تومان، پیاز سفید ۱۴ هزار تومان، پیاز زرد ۱۲ هزار تومان، سیب زمینی ۱۰ تا ۱۳ هزار تومان و گوجه فرنگی ۵ تا ۱۰ هزار تومان بوده است.',
    'بیت‌کوین با رشد بیش از ۱ درصد به قیمت ۱۶ هزار و ۶۵۰ دلار و اتریوم با افزایش حدود ۴ درصدی به قیمت هزار و ۲۰۱ دلار معامله می‌شوند.',
    'ساینا هم ۲ میلیون افزایش قیمت داشت. قیمت ساینا با این افزایش به ۲۲۵ میلیون تومان رسید.',
    'هر گرم طلای ۲۴ عیار ۲ میلیون و ۲۹ هزار و ۹۴۰ تومان قیمت گذاری شد.',
    'نرخ سکه امامی در بازار امروز با افزایش قیمتی که پیدا کرد از نیمه کانال ۱۶ میلیون عبور کرد و به رقم بی‌سابقه‌ ۱۶میلیون و ۶۰۴ هزار تومان رسید.', # ***
    'خدمات کوتاهی مو در آرایشگاه‌های زنانه طبق تعرفه اتحادیه تا ۸۰ هزار تومان هزینه دارد.', # ***
    'هر عدد کاشت مژه در شب‌های عید به ۱۰۰ تا ۲۰۰ ‌هزار تومان رسید.'  # ***
    'کوتاهی مو در آرایشگاه‌های زنانه طبق تعرفه اتحادیه تا ۱۰۰ هزار تومان هزینه دارد.', # ***
    'نرخ زیرشویی برای پراید، تیبا، پژو ۲۰۶ و ام وی ام ۲۰ هزار تومان، پژو ۳۰ هزار تومان، تویوتا کمری، سوناتا، ماکسیما و ماشین های سدان ۲۲ هزار تومان است.', # ***
    'کانون صرافان امروز  قیمت درهم را ۹ هزار و ۲۴۲ تومان برای خرید و ۹ هزار و ۳۹۷ تومان برای فروش اعلام کرده است.', # ***
    'قیمت لیر امروز ۳ آذر در بازار ارز تهران به هزار و ۹۱۰ تومان رسید.',
    'هزینه ی یک سرویس سه ساعته ی باربری درون شهری با خاور 300,000 تومان می باشد.',
    'به گزارش اقتصاد آنلاین، بر اساس آخرین نرخنامه اتحادیه میوه و تره بار تهران، قیمت عمده هر کیلوگرم انار ۱۰ تا ۴۳ هزار تومان و هر کیلوگرم انگور بی‌دانه زرد ۲۰ تا ۴۰ هزار تومان می باشد.', # When price in a range
    'در میادین میوه و تره بار قیمت هر کیلوگرم خرمالو شمال ۸ تا ۱۸ هزار تومان و خرمالوی کن ۱۵ تا ۳۷ هزار تومان عرضه می شود.', # When there is one unit for two price.,
    'شروع قیمت اصلاح سر مردانه از 40,000 تومان است.',
    'در شرکت اسپارد، قیمت اصلاح سر موی آقایان 100 هزار تومان است.',
    'در نتیجه این افت، اتریوم به زیر سطح حمایت ۱۱۸۰ دلار سقوط کرد.',
    'برای خرید یک قواره زمین کوچک با مساحت حدود ۱۰۰ مترمربع در بخش‌‌‌هایی از مناطق ۱۳ و ۱۴ که در زمره مناطق جنوبی اما پرتقاضا در بازار مسکن به شمار می‌‌‌آیند، باید رقمی نزدیک ۱۰ میلیارد تومان پرداخت کرد.',
    'بررسی قیمت‌ها نیز نشان می‌دهد که در حال حاضر قیمت یخچال ساید بای ساید ایرانی در بازار ۲۷ تا ٥٦ میلیون تومان است.',
    'پژو ۲۰۶ تیپ ۲ مدل ۱۴۰۱ به ۳۳۱ میلیون تومان رسیده‌است.',
    'بیت کوین در اوایل روز جمعه به پایین ترین حد روزانه یعنی ۱۶۳۸۸.۴۰ دلار سقوط کرد',
    'در منطقه ۱۶ محله جوادیه نیز یک آپارتمان ۵۴ متری هفت ساله دارای امکانات کامل شامل آسانسور، انباری و بالکن ۹۰۰ میلیون تومان به فروش می‌رسید اما در حال حاضر شاید با این نقدینگی بتوان به سختی آپارتمان بالای ۲۰ سال در دو منطقه‌ی مذکور خریداری کرد.',
    'قیمت تندر ۹۰ پلاس اتوماتیک امروز شنبه پنجم آذر با رشد پنج میلیونی به ۶۹۵ میلیون تومان رسید.',
    'قیمت مصوب مرغ به حدود ۵۰هزار تومان رسید، اما به دلیل مازاد عرضه، با قیمتی در حدود ۳۶ تا ۴۰ هزار تومان به دست مصرف‌کننده رسید.'
]

In [ ]:
len(samples)

84

# Part 1 - Product Name Extractor

In [ ]:
from typing import Tuple
import regex as re

<div dir=rtl>

در این قسمت قصد داریم تا با استفاده از قابلیت‌های regex، نام محصول را پیدا کنیم. برای پیدا کردن نام محصول سعی شده است تا با جست و جو در منابع مختلف، از هر گروه کلمات مربوطه انتخاب و در عبارت منظم گنجانده شود. در عین حال سعی شده تا از over engineering جلوگیری شود. چرا که over engineering لزوما منجر به دقت بالاتر نشده و از طرفی زمان و انرژی زیادی از تیم می‌برد. برای مثال در دیتاست مربوط به سایت دیجی کالا، بیشتر نام کالاها نام های طولانی و بسیار خاص می‌باشند. مثلا "بسته ی ۵ عددی کش موی سر"، و یا مواردی از این قبیل، که قرار دادن آن در مدل خیلی موثر به نظر نمی آمد.

در اینجا برای گروه‌های مختلف، کلمات مربوط به آن نوشته شده. برای گروه خودرو نیز یک الگو نوشته شده که اگر قبل از نام خودرو، خود کلمه "خودرو" و یا "ماشین" ظاهر شده بود و یا بعداز نام ماشین، مدل آن نیز بیان شده بود، بخشی از نام محصول باشد:

</div>


```python
self._car_synonyms = r'خودرو|ماشین|خودروی'
self._cars_pattern =  f'(({self._car_synonyms})\s)?({self._cars})( مدل \d+)?'
```

<div dir=rtl>

همچنین برای مسائل آموزشی مانند آموزش زبان یا نقاشی نیز، الگویی نوشته شده که اگر قبل از نام رشته آمورشی، کلمات "آموزش" و یا "تدریس" قرار گرفت بخشی از سرویس ارائه شده می‌باشد. تعداد این کلمات حدود ۷۷۰ می‌باشد. 

گاهی اوقات ممکن است کلمه‌ی موجود در rule، بخشی از یک کلمه‌ی دیگر در متن باشد. مثلا رب و ارباب:

ارباب دستور داده تا هر گوسفند کیلویی ۴۰۰ هزار تومان به فروش برسد.

در اینجا رب هم به عنوان نام محصول برمیگردد. برای جلوگیری از این کار یک الگو نوشته شده که فقط در صورتی که قبل از بعد از کلمه موجود در rule، فاصله باشد و یا یک طرف ابتدای جمله و طرف دیگر فاصله باشد را match کند.

</div>



```python
pattern = re.compile(f' ({self._all_names}) |^({self._all_names}) ')
```
<div dir=rtl>

همچنین برای در حالتی که چند محصول match شوند و یکی بخشی از دیگری باشد، محصول بزرگتر را در نظر میگیریم. مثلا برای اگر در متن، کلمه "پیاز قرمز" نوشته شود و در rule ها هم "پیاز" داشته باشیم و هم "پیاز قرمز" و هر دو match شوند، "پیاز قرمز" به عنوان اسم محصول برگردانده می‌شود.

ایده‌های دیگری برای بالا بردن دقت استخراج نام محصول زده شده که در بخش مدل نهایی (Final Mode) به آن پرداخته می‌شود.
<div>


 

In [ ]:
cars_list = [
    'آئودی','آستون مارتین','آلفا رومئو','اوپل','اویکو','اسکودا','ایسوزو','بنتلی','بوگاتی','امویام','بیسو','بی ام و','پورشه','گریتوال','جیامسی','جیلی','پونتیاک',
    'پورش','تویوتا','جگوار','جاگوار','جیپ','دانگفنگ','بیامو','چری','رنو','رولزرویس','سیناد','سوزوکی','آلفارومئو','سیتروئن','گکگونو','بایک','اپل','اندرور','شورلت',
    'شورولت','فراری','فوتون','فورد','بیوک','راین','بسترن','فولکس','بیوایدی','کوپا','برلیانس','سئات','فاو','فولکس واگن','هنتنگ','فیات','هاوال','کادیلاک','زوتی',
    'دلیکا','الدزمبیل','لامبورگینی','روور','لندمارک','لندروور','لکسوس','مازراتی','مزدا','بنز','مرسدس بنز','میتسوبیشی','نیسان','هوندا','هیوندا','هیوندای','ولوو','ام وی ام','جاماس','جک','سوبارو','آمیکو','سانگیانگ',
    'امجی','ام جی','تریومف','لادا','دوج','کرایسلر','لندروور','کوییک ','لاندروفر','۲۰۶','پژو ۲۰۶','۲۰۷','پژو ۲۰۷','۲۰۷i','۴۰۵','پژو ۴۰۵',
    '۱۱۱','۱۳۲','۱۴۱','۳۰۱','۵۰۸','۲۰۰۸',
    'سانتافه','تیبا','پراید','آریو','سابرینا','سایپا','ساینا','کوییک','زانتیا','ریو','مگان','بریلیانس','ماکسیما','پرشیا','سراتو','چانگان','رانا','رانا پلاس','دنا','زامیاد','دنا پلاس','آریسان',
    'اسمارت','تارا','سمند','سورن','پارس','پیکاپ','کپچر','کیزاشی','ویتارا','پیکان','دانگ فنگ','کاپرا','وستفیلد','اینفینیتی','هافیلوب','لینکلن','لیفان','مینیماینر','پروتون',
    'کاکی','پاژن','دیاس','هایما','هامر','دوو','آریسا','۲۰۶ تیپ دو','۲۰۶ تیپ ۵','۲۰۶ SD','پژو ۲۰۶ SD',
    'کوییک معمولی','کوییک R','کی ام سی', 'KMC', 'kmc','تندر ۹۰','سورنتو',
    'پژو ۲۰۶ تیپ ۵', 'پژو ۲۰۶ تیپ دو', 'پژو ۲۰۶ تیپ ۲',
    'ماشین', 'خودرو'
    ]

In [ ]:
class ProductNameExtractor:
  def __init__(self):
    self._wealth = 'نفت|پوند|سکه ی تمام بهار آزادی|سکه تمام بهار آزادی|سکه تمام بهار آزادی طرح جدید|سکه ی تمام بهار آزادی طرح جدید|نیم سکه|درهم|طلا ۱۸ عیار|طلا ۲۴ عیار|زمین|طلای ۲۴ عیار|اونس طلا|لیر|یورو|سکه بهارآزادی|آپارتمان|خانه|خانه ویلایی|خانه ی ویلایی|طلا|سکه|دلار|بیتکوین|بیت کوین|اتریوم|تتر|دوج|دوج کوین|شیبا|کاردانو|ترا|طلای ۱۸ عیار|طلای۱۸عیار|طلای۱۸ عیار|طلای ۱۸عیار|طلای۲۴عیار|طلای۲۴ عیار|طلای ۲۴عیار|سکه امامی|سکه ی امامی|سکه ی بهار آزادی|سکه بهار آزادی|سکه ی بهارآزادی'
    self._cars = r'|'.join(cars_list)
    self._home_products = r'یخچال|لباسشویی|ظرفشویی|هود|گاز|اجاق|فر|پادری|کمد|ملحفه|تلویزیون|پادری|آینه|پرده|تابلو|اتو|اکواریوم|قاشق|چنگال|کارد|پارچ|کاسه|بطری|لیوان|ماگ|بشقاب|لامپ|لوستر|چراغ|کلمن|فلاسک|کتری|قوری|ساعت دیواری|رومیزی|شمع|گلدان|جاروبرقی|جارو شارژی|دیگ|قندان|قابلمه|دیس|سینی|گلدان|مجسمه|گلیم|فرش|موکت|اتو|سشوار|ویلچر|قوری|کتری|کلمن|کتری|کولر|سبد|پنکه|تلفن|آباژور|گاو صندوق|تصفیه هوا|ماشین ظرفشویی'
    self._farhangi_products = r'کتاب|مجله|نرم افزار|بازی|کنسول|کیف|کوله پشتی|جامدادی|دفتر|خودکار|روان نویس|مداد|مداد رنگی|کاغذ|میز'
    self._music_instrument = r'گیتار|ارگ|پیانو|درام|پرکاشن|دف|ویولن'
    self._sanie_dasti = r'سفال|ترمه|منبت|خاتم'
    self._sports = 'چتر|چاقو|قمقمه|چادر|زیرانداز|عصا|راکت|دمبل|طناب|بارفیکس|اسکوتر|اسکیت|توپ|تور'
    self._digital_products = 'کامپیوتر|لپ تاپ|مانیتور|کیس|ماوس|کیبورد|مودم|باتری|تبلت|فکس|پرینتر|تلفن|ایکس باکس|پلی استیشن|گوشی|کیف|کاور|کیف گوشی|کاور گوشی|پاور بانک|پایه نگهدارنده گوشی|گوشی|موبایل|واقعیت مجازی|مچ‌بند|ساعت|هدفون|هدست|هندزفری|اسپیکر|هارد|فلش'
    self._car_products = 'دیسک|کمک فنر|صفحه کلاچ|لاستیک|لنت ترمز|روغن موتور|ضد یخ|تایر'
    self._other_products = 'بسته های فرهنگی|چاقو|قیچی|شیرآلات|کمپرسور|موتور برق|فرز|پیچ گوشتی|دریل|بذر|تبر|بیل|کلنگ|خاک|دستبند|حلقه|انگشتر|گوشواره|گردنبند|کود|ریکا|مایع|پاک کننده|نرم کننده|کیسه زباله|نایلون|دستکش|اسکاج|پوشک|دستمال|حوله|ادکلن|اسپری|عطر|جوراب|وان|پیشبند|پازل|لگو|عروسک|تفنگ|اسپینر|موتور|سه چرخه|دوچرخه|تاب|سرسره|ترازو|تب سنج|دما سنج|محافظ|مبلمان|تشک|پتو|بالش|کالسکه|کریر|فشار سنج|آفت کش|پمپ تخلیه|زباله|مبادلات تجاری'
    self._clothes = 'کفش|سوییشرت|هودی|شلوار|ژاکت|کمربند|پالتو|بارانی|کاپشن|پلیور|سرهمی|کت|شال|روسری|زیورآلات|عینک|کوله پشتی'

    self._foodstuffs = 'ذرت|برنج|نان|روغن|قند|شکر|سس|رب|خیارشور|ترشیجات|ترشی|کنسرو|آبلیمو|آبغوره|سرکه|ماکارونی|پاستا|رشته|زعفران|زرشک|حبوبات|غلات|قند|سویا|لوبیا|عدس|لپه|نخود|ماش|کنجد|مربا|عسل|عرقیات|حلوا|خرما|کره|ارده|گلاب|ادویه|نمک|فلفل|چاشنی|سوسیس|کالباس|گوشت|مرغ|تخم مرغ|میگو|ماهی|سرلاک|تن ماهی'
    self._foodstuffs += '|' + 'شیر|ماست|دوغ|کشک|پنیر|خامه|چای|دمنوش|قهوه|آب|ماءالشعیر|نوشابه|شربت|آب میوه|سبزی|کمپوت|ژله|پودر کیک|دسر|شکلات|تافی|آبنبات|بیسکویت|ویفر|آجیل|کشمش|پسته|بادام|خشکبار|کیک|کلوچه|چیپس|پفک|اسنک|آدامس|شیرینی'

    self._frutis = 'پیاز زرد|پیاز سفید|پیاز قرمز|پیاز شیری|خرمالو|هویج|انار|آلبالو|آلو|انبه|آناناس|انجیر|انگور|آواکادو|ازگیل|ترنج|پسته|پرتقال|تمشک|تمر|بالنگ|بلوط|بلوبری|گیلاس|توت فرنگی|توت|خربزه|خیار|خرمالو|زغال‌اخته|زالزالک|زردآلو|قندق|سنجد|سیب|شاه‌توت|طالبی|گرمک|شلیل|قیسی|کیوی|گردو|گلابی|لیمو|موز|مویز|ملون|نارگیل|نارنج|نارنگی|هلو|هندوانه|گوجه|پیاز|سیر|سیب زمینی|کاهو|سیب|پرتغال|زیتون'
    self._export_goods = r'ماکارانی|ماکارونی|متانول|پلی اتیلن|کاتود|اتیلن گلیکول|آلومینیوم|روی|گوگرد|پوست|سنگ پا|شیرین بیان|توتون|تنباکو|بستنی|شکلات|رادیو|خاویار|کتیرا|زیره|سنگ آهن|مرمر|گچ|سرامیک|شن|ماسه|سیمان|استرانسیوم|کرومیت|مس|خاک سرخ|فولاد|پروپان|پلاستیک|کود|گندم|جو|یونجه|تخمه'
    self._medicine = r'قرص اورسودزوکسی کولیک|کدئین|قرص سرماخوردگی|استامینوفن|نوافن|آزویترومایسین|وتیروکسین|اورسودزوکسی'
    self._flowers = r'رز|نرگس|اقاقیا|کاکتوس|لاله|ارکیده|آفتابگردان|نیلوفر|دالیا|زنبق|میخک|داودی|داوودی|کاملیا|عنبیه|یاس|پونی|مگنولیا|اسطوخدوس|آلاله|استوک|لیسانتوس|بواردیا|مارگریت|شقایق|بنفشه|ختمی|یاسمن|ژربرا|لیلیوم|مریم'


    self._car_synonyms = r'خودرو|ماشین|خودروی'
    self._cars_pattern =  f'(({self._car_synonyms})\s)?({self._cars})( مدل \d+)?'

    self._all_products =  f'{self._wealth}|{self._cars_pattern}|{self._home_products}|{self._farhangi_products}|{self._music_instrument}|{self._sanie_dasti}|{self._sports}|{self._digital_products}|{self._car_products}|{self._other_products}|{self._clothes}|{self._foodstuffs}|{self._frutis}|{self._export_goods}|{self._medicine}|{self._flowers}'

    self._home_services = r'نظافت|ضدعفونی|پذیرایی|سمپاشی|باغبانی|مبل شویی|طراحی و ساخت مبل|تعمیر صندلی اداری|رویه کوبی|دوخت کاور|تعمیر ساعت|برش لیزری|تعمیر دوچرخه|تعمیر کولر|تعمیر شوفاژ|تعمیر آبگرمکن|تعمیر شومینه|نصب شومینه|لوله کشی گاز|لوله کشی آب|لوله کشی فاضلاب|نصب سیفون|نصب فلاش تانک|نصب روشویی|نصب سینک|تخلیه چاه|لوله بازکنی|حفر چاه|لایروبی|نصب دوربین|سیم کشی|سیم پیچی|تعمیر دزدگیر|نصب دزدگیر|قالیشویی|خشکشویی|نماشویی|کفسابی'
    self._digital_services = r'تعمیر یخچال|تعمیر ماشین لباسشویی|تعمیر لپ تاپ|تعمیر رایانه|تعمیر کامپیوتر|تعمیر تلویزیون|تعمیر لوازم برقی|تعمیر لوزم صوتی و تصویری|تعمیر اتو|تعمیر رادیو|تعمیر غذاساز|تعمیر پرینتر|شارژ کارتریج'
    self._cosmetic_services = r'کوتاهی مو|بافت مو|اکتنشن مو|شینیون مو|آمبره|سامبره|کراتینه|فر دائم|رنگ مو|بالیاژ|کاشت مژه|سایه ابرو|میکاپ|بوتاکس|کاشت ناخن|ترمیم ناخن|اصلاح سر|کاشت ریش|کاشت مو|لیزر'
    self._building_services = r'بنایی|گچبری|کاشی کاری|جوشکاری|شیشه بری|دیوار کشی|کانال کشی|درب ضدسرقت|درب ضد سرقت|نصب سرامیک|سیمان کاری|نجاری|ایزوگام|آسانسور|اسباب کشی|باربری'
    self._carwash_services = r'زیرشویی|کلیدسازی|کارواش|صفرشویی|موتورشویی|موتور شویی|سرامیک|کارواش بخار|واکس|شفاف سازی'
    self._other_services = r'هتل|تاکسی|آژانس|جت اسکی|چاپ|فتوکپی|تحقیق'
    self._educational_services = r'آموزش زبان|(زبان)* انگلیسی|(زبان)* عربی|(زبان)* ترکی|(زبان)* ترکی|(زبان)* فرانسوی|دف|نقاشی|طراحی|عکاسی'
    self._educational_pattern = f'(آموزش|تدریس)* ({self._educational_services})'

    self._all_services = f'{self._home_services}|{self._digital_services}|{self._cosmetic_services}|{self._building_services}|{self._carwash_services}|{self._other_services}|{self._educational_pattern}'

    self._all_names = f'{self._all_products}|{self._all_services}'

  def run(self, input_str):
    pattern = re.compile(f' ({self._all_names}) |^({self._all_names}) ')
    
    iterator = list(re.finditer(pattern, input_str))

    result = {'product_name': None, 'product_name_span': None}

    if iterator:
      result['product_name'] = iterator[0].group(0)
      result['product_name_span'] = list(iterator[0].span())

    return result
  def is_service(self, word: str) -> bool:
    if word in self._all_services:
      return True
    return False

  @property
  def rules_count(self):
    return len(self._all_names.split('|'))

    def _get_pos_of_each_word(sentences_dictionaries, word):
      for sentence_dictionaries in sentences_dictionaries:
        for dictionary in sentence_dictionaries:
          if dictionary['text'] == word:
            return dictionary['pos']


#### Number of rules

In [ ]:
ProductNameExtractor().rules_count

773

## Number extractor

<div dir=rtl>
<big>
این قسمت از کد متناظر با بخش number extractor از کتابخانه parsi.io می باشد که بعضی از قسمت های آن را که در ادامه به آنها اشاره می کنیم تغییر داده ایم.
</br>
</big>
</div>

### pattern.py

<div dir=rtl>
<big>
در کد این کتابخانه فرض بر این است که اعداد فقط شامل یک علامت جداکننده (. یا ,  ) هستند که نشان دهنده علامت اعشار است اما در داده هایی که ما برای استخراج قیمت و خدمت جمع آوری کردیم اعدادی مانند ۱,۴۹۴,۴۰۰ وجود داشتند که نه تنها بیش از یک علامت جدا کننده داشتند بلکه این علامت جداکننده برای آن ها به معنی اعشار نبوده و صرفا برای جدا کردن بخش های مختلف قیمت استفاده شده است .
</br>
برای رفع این مشکل و استخراج اعداد به صورت کامل PATTERN_NUMBER_WITH_DIGITS  را به گونه ای باز نویسی کردیم که  برای حالت  بیش از یک علامت جداکننده نیز درست کار کند.  
</br>
</big>
</div>

```python
PATTERN_NUMBER_WITH_DIGITS = f'{PATTERN_DIGIT}*{PATTERN_DOT}?{PATTERN_DIGIT}+{PATTERN_DOT}?{PATTERN_DIGIT}*'
```


In [ ]:
def join_patterns(items):
	return '(?:' + '|'.join(list(items)) + ')'  # No capture grouping


# Let's ignore this for now
# def accept_all_ya(pattern: str):
ALL_YA = '[یي]'  # Both ی in perso-arabic script
# 	return pattern.replace('ی', ALL_YA)

ENGLISH_NON_ZERO_DIGITS = '123456789'
PERSIAN_NON_ZERO_DIGITS = '۱۲۳۴۵۶۷۸۹'
ENGLISH_ZERO_DIGIT = '0'
PERSIAN_ZERO_DIGIT = '۰'
DIGIT_NEG = '-'

PERSIAN_V = 'و'
PERSIAN_NEG = 'منفی'

NIM_SPACE = '\u200c'
WHITE_SPACE = rf'[{NIM_SPACE}\s]'

PERSIAN_SPECIAL_CASES = {
	'صفر': 0,
	#'هر': 1,
}

PERSIAN_UNDER_10_NUMBERS = {
	'یک': 1,  # Zero has different properties
	'دو': 2,
	'سه': 3,
	'سوم': 3,
	'چهار': 4,
	'پنج': 5,
	'شش': 6,
	'هفت': 7,
	'هشت': 8,
	'نه': 9,
	'نیم': 0.5
}

PERSIAN_UNDER_20_NUMBERS = {
	'ده': 10,
	'یازده': 11,
	'دوازده': 12,
	'سیزده': 13,
	'چهارده': 14,
	'پانزده': 15,
	'شانزده': 16,
	'هفده': 17,
	'هجده': 18,
	'نوزده': 19
}

TEXT_SI = 'سی' # special case with 100
TEXT_SAD = 'صد'
PERSIAN_UNDER_100_NUMBERS = {
	'بیست': 20,
	'سی': 30,
	'چهل': 40,
	'پنجاه': 50,
	'شصت': 60,
	'هفتاد': 70,
	'هشتاد': 80,
	'نود': 90,
}

PERSIAN_UNDER_1000_NUMBERS = {
	'صد': 100,
	# 'یکصد': 100,
	'دویست': 200,
	# 'سیصد': 300,
	# 'چهارصد': 400,
	'پانصد': 500,
	# 'ششصد': 600,
	# 'هفتصد': 700,
	# 'هشتصد': 800,
	# 'نهصد': 900,
	'هزار': 1000,
	# 'یکهزار': 1000
}


PERSIAN_REQUIRED_EXTENDABLE_NUMBERS = {
    'دهم': 0.1,
   	'صدم': 0.01,
   	'هزارم': 0.001,

	'صد': 100,  # YES I know these are redundant but it has a different usage
	'هزار': 1000,
	'میلیون': 10**6,  # Handle different form?
	'ملیون': 10**6,
	'میلیارد': 10**9
}

# Handling 123, ۲۱۳, 3۲1
PATTERN_DIGIT = join_patterns(list(ENGLISH_NON_ZERO_DIGITS + PERSIAN_NON_ZERO_DIGITS + ENGLISH_ZERO_DIGIT + PERSIAN_ZERO_DIGIT))
PATTERN_DOT = '(?:\.|٫)'
DOT_LIST = ['.', '٫']

# PATTERN_NUMBER_WITH_DIGITS = f'{PATTERN_DIGIT}*{PATTERN_DOT}?{PATTERN_DIGIT}+'
PATTERN_NUMBER_WITH_DIGITS = f'{PATTERN_DIGIT}*{PATTERN_DOT}?{PATTERN_DIGIT}+{PATTERN_DOT}?{PATTERN_DIGIT}*'
# Handling یگ, یازده, دویست
ALL_PERSIAN_SIMPLE_NUMBERS = { # Order is important :|
	**PERSIAN_UNDER_1000_NUMBERS,
	**PERSIAN_UNDER_100_NUMBERS,
	**PERSIAN_UNDER_20_NUMBERS,
	**PERSIAN_UNDER_10_NUMBERS
}
PATTERN_SIMPLE_NUMBER = join_patterns(ALL_PERSIAN_SIMPLE_NUMBERS.keys())
# Handling 33 هزار
PATTERN_BEFORE_EXTEND = join_patterns([rf'{PATTERN_NUMBER_WITH_DIGITS}{WHITE_SPACE}*', rf'{PATTERN_SIMPLE_NUMBER}{WHITE_SPACE}*'])
PATTERN_EXTEND = join_patterns(PERSIAN_REQUIRED_EXTENDABLE_NUMBERS)
PATTERN_EXTENDABLE_NUMBER = rf'{PATTERN_BEFORE_EXTEND}{PATTERN_EXTEND}(?:{WHITE_SPACE}+{PATTERN_EXTEND})*'
# Handing number with و
# REASON of using dict: cleaner code when using get_value in the main
ALL_PARTS = {
	'EXTENDED': PATTERN_EXTENDABLE_NUMBER,
	'DIGIT': PATTERN_NUMBER_WITH_DIGITS,
	'SIMPLE': PATTERN_SIMPLE_NUMBER
}  # Order is important :|
PATTERN_SINGLE_NUMBER = join_patterns(ALL_PARTS.values())  # We do not need the keys here
PATTERN_MULTIPLE_NUMBER = rf'{PATTERN_SINGLE_NUMBER}(?:{WHITE_SPACE}+{PERSIAN_V}{WHITE_SPACE}+{PATTERN_SINGLE_NUMBER})*'

ALL_NEGS = join_patterns([rf'{PERSIAN_NEG}{WHITE_SPACE}+', rf'{DIGIT_NEG}{WHITE_SPACE}*'])
PATTERN_ALL_NUMBER_EXCEPT_ZERO = f'{ALL_NEGS}?{PATTERN_MULTIPLE_NUMBER}'
PATTERN_ALL_NUMBER = join_patterns([PATTERN_ALL_NUMBER_EXCEPT_ZERO] + list(PERSIAN_SPECIAL_CASES.keys()))

BEFORE_NUMBER = r'(?:\W|^)'

END_WORD_LIST = [
	'م',
	'مین',
	'بار',
	'عدد',
	'برابر'
]
END_WORD_LIST_WITH_AFTER = [f'{item}{ALL_YA}?{WHITE_SPACE}' for item in END_WORD_LIST]
AFTER_NUMBER = join_patterns(END_WORD_LIST_WITH_AFTER + [r'\W', '$'])

PATTERN_SEARCH = BEFORE_NUMBER + f'({PATTERN_ALL_NUMBER})' + AFTER_NUMBER

### number_extractor.py

<div dir=rtl>
<big>
در این قسمت  تابع  __get_value_digit نیز نیاز به تغییر دارد چرا که در این کد فرض بر این است که علامت جداکننده نشان دهنده اعشار است و عدد با دو علامت جداکننده نامعتبر است .
</br>
به همین منظور فرض کردیم که این علامت های جداکننده برای اعدادی که دارای بیش از یک علامت جداکننده هستند فقط برای نشان دادن بخش هایی از قیمت است و می توان آن را حذف کرد.
</big>
</div>


<div dir=rtl>
<big>
</br> 
البته ما در این پروژه به این قسمت از کد نیازی نداریم و فقط از span ای که این مدل برای قیمت خروجی می دهد در بخش بعد استفاده می کنیم.
</big>
</div>

In [ ]:
import typing
import math
import re

class NumberExtractor:
	def __init__(self):
		# extract_phrases regex
		self.re_search = re.compile(PATTERN_SEARCH)
		self.re_norm_add_space = re.compile(f'({PATTERN_NUMBER_WITH_DIGITS})')
		# self.re_norm_revert = re.compile(f' ?({PATTERN_NUMBER_WITH_DIGITS}) ?')
		# get_value_extend regex
		self.re_before_extend = re.compile(PATTERN_BEFORE_EXTEND)
		self.re_extend = re.compile(PATTERN_EXTEND)
		# get_sub_phrase_value regex
		self.re_parts = {}
		for key, val in ALL_PARTS.items():
			self.re_parts[key] = re.compile(val)
		# get_value regex
		self.re_neg = re.compile(ALL_NEGS)
		self.re_single = re.compile(PATTERN_SINGLE_NUMBER)

	def __find_index_match(self, base, normed):
		base_idx = [idx for idx, c in enumerate(base) if c != ' ']
		normed_idx = [idx for idx, c in enumerate(normed) if c != ' ']
		return dict(list(zip(normed_idx, base_idx)))

	def __extract_spans(self, input_sentence: str) -> typing.List[typing.Tuple[int, int]]:
		return_value = []
		normed_input_sentence = self.re_norm_add_space.sub(r' \1 ', input_sentence)  # adding space around digits
		span_convert_dict = self.__find_index_match(input_sentence, normed_input_sentence)
		for match in self.re_search.finditer(normed_input_sentence):
			_span_b, _span_e = match.span(1)
			span = span_convert_dict[_span_b], span_convert_dict[_span_e - 1] + 1
			return_value.append(span)
		return return_value

	def __get_value_digit(self, sub_phrase: str) -> typing.Tuple[float]:
		for dot in DOT_LIST:
			if sub_phrase.count(dot) > 1 or sub_phrase.count(dot) > 1:
				sub_phrase = sub_phrase.replace('.', '')
				sub_phrase = sub_phrase.replace('٫', '')
		intab = PERSIAN_ZERO_DIGIT + PERSIAN_NON_ZERO_DIGITS + '٫'
		outtab = ENGLISH_ZERO_DIGIT + ENGLISH_NON_ZERO_DIGITS + '.'
		translation = str.maketrans(intab, outtab)
		return float(sub_phrase.translate(translation)),

	def __get_value_simple(self, sub_phrase: str) -> typing.Tuple[float]:
		return ALL_PERSIAN_SIMPLE_NUMBERS[sub_phrase],

	def __get_value_extend(self, sub_phrase: str) -> tuple:
		multiplier = 1

		before_extend = self.re_before_extend.findall(sub_phrase)[0]
		before_extend_value = self.__get_value(before_extend)  # a digit_Value or a simple value
		sub_phrase = self.re_before_extend.sub('', sub_phrase, 1)  # remove before extend part

		extends = self.re_extend.findall(sub_phrase)

		if before_extend.strip() == TEXT_SI and extends[0] == TEXT_SAD: # 300 special case
			before_extend_value = 3

		for extend in extends:
			multiplier *= PERSIAN_REQUIRED_EXTENDABLE_NUMBERS[extend]

		if multiplier < 1:
			return before_extend_value * multiplier,

		if extends[0] == TEXT_SAD:
			return before_extend_value * multiplier,

		return before_extend_value, multiplier

	def __get_sub_phrase_value(self, sub_phrase: str) -> tuple:
		get_value_mapper = {
			'EXTENDED': self.__get_value_extend,
			'DIGIT': self.__get_value_digit,
			'SIMPLE': self.__get_value_simple
		}
		for sub_phrase_kind, re_obj in self.re_parts.items():
			if re_obj.match(sub_phrase):
				return get_value_mapper[sub_phrase_kind](sub_phrase)

	def __get_value(self, phrase: str) -> float:
		return_value = 0
		temp_value = 0
		highest_extend_multiplier = 0
		negative_number = False

		if phrase in PERSIAN_SPECIAL_CASES:
			return PERSIAN_SPECIAL_CASES[phrase]
		if len(self.re_neg.findall(phrase)):
			negative_number = True

		sub_phrases = self.re_single.findall(phrase)
		sub_phrase_values = map(self.__get_sub_phrase_value, sub_phrases)

		for index, value in enumerate(sub_phrase_values):
			addition = value[0]

			if temp_value != 0:
				if temp_value % 10**math.ceil(math.log(addition, 10)) >= 1:
					raise ValueError(index)  # Panic: There are more than one valid number in the phrase

			temp_value += addition

			if len(value) == 2:
				multiplier = value[1]
				if highest_extend_multiplier < multiplier:
					highest_extend_multiplier = multiplier
					temp_value += return_value
					return_value = 0
				return_value = return_value + temp_value * multiplier
				temp_value = 0

		return_value += temp_value

		if negative_number:
			return_value = -return_value

		return return_value

	def run(self, input_sentence):
		return_value = []
		spans = self.__extract_spans(input_sentence)
		i = 0
		while i < len(spans):
			span = spans[i]
			phrase = input_sentence[span[0]: span[1]]
			try:
				return_value.append({
					'span': list(span),
					'phrase': phrase,
					'value': self.__get_value(phrase)
				})
				i += 1
			except ValueError as exp:  # Handling __get_value panic
				last_phrase_parts = phrase.split(PERSIAN_V, exp.args[0])
				phrase1_end = len(phrase) - len(last_phrase_parts[-1])
				_sub_span1 = self.__extract_spans(phrase[:phrase1_end])[0]
				_sub_span2 = self.__extract_spans(last_phrase_parts[-1])[0]
				sub_span1 = _sub_span1[0] + span[0], _sub_span1[1] + span[0]
				sub_span2 = _sub_span2[0] + span[0] + phrase1_end, _sub_span2[1] + span[0] + phrase1_end
				spans[i] = sub_span1
				spans.insert(i + 1, sub_span2)
		return return_value




### test number extractor 

In [ ]:
extractor = NumberExtractor()
for i in range(len(samples)):
    # add these two lines to preprocces
    samples[i] = samples[i].replace("/", ".")
    samples[i] = samples[i].replace(",", "٫")
    print("sample " + str(i) + " : ")
    print(samples[i])
    print(extractor.run(samples[i]))

sample 0 : 
برنج سر لاشه گلچین امساله اعلاء ۱۰ کیلویی (تضمین کیفیت) هر کیسه با قیمت یک میلیون تومان به بازار عرضه می شود.
[{'span': [32, 34], 'phrase': '۱۰', 'value': 10.0}, {'span': [72, 81], 'phrase': 'یک میلیون', 'value': 1000000}]
sample 1 : 
قیمت روغن مخصوص سرخ کردنی بدون پالم اویلا ۱.۸ کیلوگرمی ، هر بطری ۲۰۰ هزار تومان می باشد.
[{'span': [42, 45], 'phrase': '۱.۸', 'value': 1.8}, {'span': [65, 73], 'phrase': '۲۰۰ هزار', 'value': 200000.0}]
sample 2 : 
روغن حیوانی گاوی اعلاء نیم کیلویی با قیمت یکصد هزار تومان به دست مشتری می رسد.
[{'span': [23, 26], 'phrase': 'نیم', 'value': 0.5}, {'span': [42, 51], 'phrase': 'یکصد هزار', 'value': 100000}]
sample 3 : 
ماکارانی زر ماکارون قطر 1.5 وزن 700 گرم با قیمت ۱۵ هزار تومان برای هر بسته در فروشگاه های کشور توزیع می شود.
[{'span': [24, 27], 'phrase': '1.5', 'value': 1.5}, {'span': [32, 35], 'phrase': '700', 'value': 700.0}, {'span': [48, 55], 'phrase': '۱۵ هزار', 'value': 15000.0}]
sample 4 : 
نان حلوایی مجلسی کاسه ای بسته 60 عددی مناسب جهت انو

# Part 2 - Price and Unit price Extractor

<div dir=rtl>
<big>
 این قسمت شامل مجموعه الگوهایی است که در بخش بعدی مورد استفاده قرار می گیرند .
</big>
</div>
</br>



---


<div dir=rtl>
PATTERN_UNIT_PRICE
</br>
این الگو نشان دهنده تمام واحدهای پول است که ممکن است در متن ورودی داشته باشیم.
</div>


---


<div dir=rtl>
LEN_LONGEST_UNIT_PRICE
</br>
در این کد بعد از استخراج اعداد در متن،‌ بررسی می شود که آیا عدد بدست آمده متناظر با قیمت است یا نه. برای این کار به اندازه LEN_LONGEST_UNIT_PRICE بعد از این عدد را بررسی می کنیم تا  در صورت تطبیق آن با یکی از واحد های الگو قبلی، این عدد را به عنوان یک عددی که به قیمت اشاره می کند در نظر بگیریم.
</div>

---

<div dir=rtl>
PATTERN_PRICE_AMOUNT_WITHOUT_UNIT
</br>
بعضی از اعداد هستند که ممکن است به قیمت اشاره کنند اما واحد بعد از آنها نداشته باشیم. برای مثال در بعضی از موارد واحد بعد از اعداد میلیون و میلیارد گفته نمی شود.
در این حالت از سایر کلماتی که در قبل و بعد از این عدد می آید استفاده می کنیم تا ببنیم آیا می توان گفت این عدد قیمت است یا نه.
</div>

---

<div dir=rtl>
PATTERN_AFTER_PRICE, PATTERN_ABOUT_PRICE
</br>
این دو الگو به حالاتی اشاره می کنند که در صورتی که در قبل یا بعد از عدد بیایند می توان عدد بکار رفته در متن را (هر چند واحدی که نشان دهنده قیمت است نداشته باشد)  متناظر با یک قیمت در نظر گرفت.
</div>

---

<div dir=rtl>
LEN_LONGEST_PATTERN_ABOUT_PRICE
</br>
در صورتی که بخواهیم قبل یا بعد از عدد را برای بررسی دو الگو قبل استفاده کنیم طولی از رشته را که به اندازه این پارامتر است در نظر می گیریم.
</div>

---

<div dir=rtl>
PATTERN_RANGE_PRICE, PATTERN_RANGE_TA_Preposition, PATTERN_RANGE_BE_Preposition
</br>
در بعضی از مثال ها به جای گزارش یک قیمت برای محصول رنجی از قیمت برای آن اشاره شده است (مثلا قیمت آن بین ۲۰ هزار تومان تا ۳۰ هزار تومان است). در این حالت از الگو PATTERN_RANGE_PRICE برای رفع این چالش استفاده کردیم.
به این صورت که رشته متناظر با بین دو عدد گرفته می شود و در صورتی که این رشته معادل با یکی از حروف اضافه (تا، به) باشد هر دو عدد به عنوان یک رنج برای محصول در نظر گرفته می شوند . حالت دیگری که ممکن است پیش بیاید نیز این است که این رشته شامل واحد قیمت عدد اول نیز باشد در این صورت نیز می توان هر دو عدد را به عنوان یک رنج برای قیمت در نظر گرفت.
</div>

---

<div dir=rtl>
سایر الگو ها، ترکیب الگو هایی قبلی هستند با WHITE_SAPCE که نشان دهنده این است که قبل یا بعد از آنها می تواند تعداد دلخواهی فاصله وجود داشته باشد.
</div>

---

<div dir=rtl>
</br>
</div>

In [ ]:
# put all patterns here
PATTERN_UNIT_PRICE = 'دینار|روپیه|دلار|درهم|تومان|لیره|پوند|ریال|فرانک|روبل|یورو'
LEN_LONGEST_UNIT_PRICE = 5
PATTERN_ABOUT_PRICE = 'هزینه|پرداخت|قیمت|مبلغ|واریز' 
PATTERN_PRICE_AMOUNT_WITHOUT_UNIT = 'میلیون|میلیارد'
PATTERN_AFTER_PRICE = 'افزایش قیمت|کاهش قیمت|تغییر قیمت'
PATTERN_AFTER_PRICE_WITHOUT_UNIT = f'{WHITE_SPACE}*{PATTERN_AFTER_PRICE}{WHITE_SPACE}*'
LEN_LONGEST_PATTERN_ABOUT_PRICE = 12
WHITE_SPACE = rf'[{NIM_SPACE}\s]'
PATTERN_RANGE_TA_Preposition = 'تا'
PATTERN_RANGE_BE_Preposition = 'به'
PATTERN_RANGE_PRICE = f'{WHITE_SPACE}*({PATTERN_UNIT_PRICE}|){WHITE_SPACE}*({PATTERN_RANGE_TA_Preposition}|{PATTERN_RANGE_BE_Preposition}){WHITE_SPACE}*'

UNIT_PRICE_SEARCH = f'{WHITE_SPACE}*{PATTERN_UNIT_PRICE}{WHITE_SPACE}*'

<div dir=rtl>
<big>
در تابع run در کلاس ProductPriceExtractor ابتدا رشته ورودی را به همراه span متناظر با تمام اعداد بکار رفته در متن  دریافت می کنیم.
</br>
 پس از آن span هایی که  مطابق با الگوی PATTERN_RANGE_PRICE به یک بازه از اعداد اشاره می کنند ادغام میشوند.
</br>
در  آخر نیز بررسی می شود که کدام یک از اعداد متناظر با قیمت هستند. در ابتدا در صورتی که بعد از عدد واحد داشته باشیم، واحد و عدد را بعنوان خروجی در نظر میگیریم و در صورتی که بعد از عدد واحد نداشته باشیم، الگوهای خاصی که ممکن است در قبل یا بعد از عدد داشته باشیم که به قیمت بودن آن عدد اشاره می کند بررسی می شوند و در صورت تطبیق عدد بعنوان قیمت در نظر گرفته شده و به صورت بدون واحد در خروجی قرار می گیرد.
</big>
</div>


In [ ]:
import re
import json
import hazm
import re
from pathlib import Path

class Preprocessor:
    def preprocess(self, text):
        text = text.replace("/", ".")
        text = text.replace(",", "٫")
        text = text.replace('٥', '۵')
        text = text.replace('٦', '۶')
        # text = self.normalize(text)
        # text = self.fix_arabic_ta(text)
        # text = self.fix_arabic_ye(text)
        return text

    def normalize(self, text):
        normalizer = hazm.Normalizer()
        return normalizer.normalize(text)


    def fix_arabic_ta(self, text):
        return text.replace('ة', 'ه')

    def fix_arabic_ye(self, text):
        return text.replace('ي', 'ی')

    
class ProductPriceExtractor:
  def __init__(self):
    self.re_search_price_unit = re.compile(UNIT_PRICE_SEARCH)
    self.re_search_about_price = re.compile(PATTERN_ABOUT_PRICE)
    self.re_search_after_price = re.compile(PATTERN_AFTER_PRICE_WITHOUT_UNIT)
    self.re_search_range_price = re.compile(PATTERN_RANGE_PRICE)
    self.re_search_amount_without_unit = re.compile(PATTERN_PRICE_AMOUNT_WITHOUT_UNIT)
    # self.preprocessor = Preprocessor()


  def __find_index_match(self, base, normed):
    base_idx = [idx for idx, c in enumerate(base) if c != ' ']
    normed_idx = [idx for idx, c in enumerate(normed) if c != ' ']
    return dict(list(zip(normed_idx, base_idx)))

  def __extract_price_unit(self, after_number_str):
    return_value = []
    after_number_str = after_number_str[0: LEN_LONGEST_UNIT_PRICE]
    for match in self.re_search_price_unit.finditer(after_number_str):
      _span_s = match.span()[0]
      _span_e = match.span()[1]
      return True, after_number_str[int(_span_s): int(_span_e)]
    return False, []
  
  def __check_number(self, price_number):
    if price_number == 'هر':
      return 'یک'
    return re.sub(PATTERN_UNIT_PRICE, '', price_number)

  def __test_number_is_price(self, number, before_number, after_number):
    for match in self.re_search_amount_without_unit.finditer(number):
      for match in self.re_search_about_price.finditer(before_number):
        return True
      if self.re_search_after_price.match(after_number):
        return True
    return False

  def extract_numbers(self, input_sentence):
    extractor = NumberExtractor()
    extracted_numbers = extractor.run(input_sentence)
    num_spans = []
    num_phrases = []
    for num_dict in extracted_numbers:
      span = num_dict['span']
      num_spans.append(span)

    return num_spans
  
  def run(self, input_sentence, number_spans):
    return_value = []    
    # input_sentence = self.preprocessor.preprocess(input_sentence)
    # number_spans = self.extract_numbers(input_sentence)

    # extract range numbers
    extracted_number_spans = []
    i = 0
    while i < len(number_spans):
      # last number 
      if i == len(number_spans) - 1:
        extracted_number_spans.append(number_spans[i])
        break
      between_numbers_str = input_sentence[number_spans[i][1]:number_spans[i+1][0]]
      span = [number_spans[i][0], number_spans[i+1][1]]
      if self.re_search_range_price.fullmatch(between_numbers_str):
        extracted_number_spans.append(span)
        i += 2
      else:
        extracted_number_spans.append(number_spans[i])
        i += 1

    for num_span in extracted_number_spans:
      have_unit, unit_phrase = self.__extract_price_unit(input_sentence[num_span[1] :].lstrip())
      if have_unit:
        price_number = input_sentence[num_span[0]: num_span[1]]  
        price_number = self.__check_number(price_number)
        return_value.append({
                    'price_amount': price_number,
                    'price_unit': unit_phrase
                })
      else:
        number = input_sentence[num_span[0]: num_span[1]]  
        is_price = self.__test_number_is_price(number, input_sentence[max(0, num_span[0] - LEN_LONGEST_PATTERN_ABOUT_PRICE) :num_span[0]], input_sentence[num_span[1] :min(len(input_sentence), num_span[1] + LEN_LONGEST_PATTERN_ABOUT_PRICE)])
        if is_price:
          return_value.append({
                    'price_amount': number,
                    'price_unit': ''  # احتمالا باید تومان باشد
                })

        
    return return_value

# Part 3 - Product Unit and Product Amount Extractor

<div style="text-align: right" dir="rtl"> 
<h2><b>بخش سوم - استخراج واحد و مقدار محصول/خدمات</b></h2>
در این قسمت با استفاده از الگو هایی که در ادامه توضیح داده میشوند قسمت هایی از 
متن ک ممکن است شامل واحد و مقدار واحد محصول/خدمات باشند، استخراج میشوند.

قابل توجه است که واحد هایی که مکررا در اینگونه جملات تکرار میشوند در متغیر units به صورت دیکشنری ذخیره شده اند.
این متغیر شامل units و special units می‌شود که هر کدام لیستی از واحد ها هستند با این تفاوت که units واحد هایی هستند که یا با عدد در جمله به کار میروند یا با حروفی که نشان از مقدار موحد بودن آن واحد از محصول میباشد و special units شامل واحدهایی میشود که بدون نام بردن تعداد به هرشکل استفاده میشوند، مانند "ماهانه"

</div>

<div style="text-align: right" dir="rtl"> 
پترن های به کار رفته به شرح زیر می‌باشند: 

(پترن ۱)
</div>

```python
{START_OF_PATTERN}(?P<amount>{PATTERN_SEARCH})(?P<unit>{PATTERN_PROD_UNIT}){ALL_YA}*{END_OF_PATTERN}
```

<div style="text-align: right" dir="rtl"> 
که برای پیداکردن قسمتی از متن است که با یک عدد شروع میشود سپس بعد آن نام واحدی به کار رفته و ممکن است بعد از واحد حرف "ی" یا "یی" به کار رفته باشد یا نرفته باشد. مانند : سیب زمینی دو کیلو(یی) ۵۰ هزار تومان میباشد. 

(پترن ۲)

این پترن ها که در ادامه میبینید همه ی آن ها برای مواردی به کار میروند که نشان از موحد بودن آن واحد از محصول است به این شکل اند:
<br><br>
هر به همراه نام واحد ٬مثال: هر سال
</div>

```python
{START_OF_PATTERN}?{HAR_PATTERN}{WHITE_SPACE}(?P<unit2>{PATTERN_PROD_UNIT})({END_OF_PATTERN})
```

<div style="text-align: right" dir="rtl"> 
نام واحد + یی یا ی ٬مثال: ماهی
</div>

```python
{START_OF_PATTERN}?(?P<unit1>{PATTERN_PROD_UNIT}){ALL_YA}+({END_OF_PATTERN})
```

<div style="text-align: right" dir="rtl"> 
نام واحد + ای ٬ مثال : کیلو ای
</div>

```python
{START_OF_PATTERN}?(?P<unit3>{PATTERN_PROD_UNIT}){EI_PATTERN}({END_OF_PATTERN})
```

<div style="text-align: right" dir="rtl"> 
و در نهایت خروجی به شکل لیستی از دیکشنری هایی  نام واحد و مقدار واحد استخراج شده از متن خواهد بود.
</div>



In [ ]:
units = {'units': ['بشکه-نفت آمریکایی', 'میلی متر مکعب', 'میلی متر مربع', 'سانتیمتر مربع', 'گالن آمریکایی',
  'سانتیمتر مکعب', 'کیلومتر مربع', 'گالن انگلیسی', 'شعاع خورشیدی', 'دسی متر مکعب',
  'میکرو ثانیه', 'اونس معمولی', 'مایل دریایی', 'واحد نجومی', 'کیلو نیوتن',
  'کیلو ثانیه', 'میلی ثانیه', 'نانو ثانیه', 'پیکو ثانیه', 'فرسنگ شرعی',
  'ثانیه نوری', 'مایل مربع', 'یارد مکعب', 'اینچ مربع', 'میکرو گرم',
  'اونس تروا', 'یارد مربع', 'من لرستان', 'من اصفهان', 'اینچ مکعب',
  'میلی لیتر', 'من تبریز', 'آنگستروم', 'میلی\u200cولت', 'فوت مکعب',
  'متر مکعب',
  'سال نوری',
  'سال قمری',
  'میلی گرم',
  'فوت مربع',
  'سانتیمتر',
  'ماه قمری',
  'متر مربع',
  'دیاستول',
  'تازیانه',
  'کیلومتر',
  'پیکومتر',
  'کیلوگرم',
  'کیلوولت',
  'بارخانه',
  'مترمکعب',
  'دسی متر',
  'نانومتر',
  'میلیمتر',
  'فمتومتر',
  'دستگاه',
  'استکان',
  'فرلانگ',
  'مگاولت',
  'میکرون',
  'سیستول',
  'پیمانه',
  'کارتون',
  'خروار',
  'ثانیه',
  'انبار',
  'نیوتن',
  'فاتوم',
  'عراده',
  'سرویس',
  'صندوق',
  'اسلاگ',
  'پیاله',
  'پارسک',
  'پارچه',
  'نمونه',
  'قیراط',
  'زنجیر',
  'من ری',
  'دقیقه',
  'فروند',
  'دوجین',
  'سی سی',
  'قواره',
  'قلاده',
  'فرسنگ',
  'واقعه',
  'هکتار',
  'قراسه',
  'مثقال',
  'بطانه',
  'طغرا',
  'لنگه',
  'کیلو',
  'پرده',
  'نخود',
  'جرعه',
  'مایل',
  'لقمه',
  'خلال',
  'اصله',
  'لیتر',
  'بوته',
  'دانه',
  'لوله',
  'دهنه',
  'جعبه',
  'فرسخ',
  'صفحه',
  'بدره',
  'کاسه',
  'هفته',
  'شیشه',
  'شعله',
  'باکس',
  'قطعه',
  'درجه',
  'دسته',
  'قطره',
  'خوشه',
  'پاره',
  'گله',
  'یارد',
  'کلاف',
  'نسخه',
  'قالب',
  'واحد',
  'قبضه',
  'حلقه',
  'بهله',
  'بشکه',
  'پوند',
  'شاخه',
  'چارک',
  'رشته',
  'بطری',
  'جریب',
  'بقچه',
  'اینچ',
  'فقره',
  'بسته',
  'اونس',
  'ساعت',
  'کیسه',
  'طاقه',
  'چشمه',
  'قلمه',
  'دانگ',
  'گندم',
  'تخته',
  'قرص',
  'جلد',
  'سیخ',
  'جفت',
  'توپ',
  'رأس',
  'سکه',
  'فال',
  'دست',
  'سیر',
  'پرس',
  'برش',
  'قلم',
  'عدد',
  'مشت',
  'بته',
  'روز',
  'گرم',
  'سری',
  'سال',
  'بال',
  'بند',
  'سوت',
  'جین',
  'جام',
  'باب',
  'ذرع',
  'رام',
  'فوت',
  'بغل',
  'متر',
  'ورق',
  'ثوب',
  'عدل',
  'وجب',
  'نفر',
  'ولت',
  'برگ',
  'قرن',
  'دهه',
  'بار',
  'تن',
  'ماه',
  'میل',
  'کله',
  'سر',
  'حب',
  'من',
  'مو',
  'تا',
  'گل',
  'تن',
  'نخ',
  'گز'],
 'sp_units': ['ماهیانه', 'سالیانه']}

### Patterns

In [ ]:
persian_alpha_codepoints = '\u200c\u0621-\u0628\u062A-\u063A\u0641-\u0642\u0644-\u0648\u064E-\u0651\u0655\u067E\u0686\u0698\u06A9\u06AF\u06BE\u06CC'

EI_PATTERN = 'ای'
HAR_PATTERN = 'هر'
PATTERN_PROD_UNIT = join_patterns(units['units'])
END_OF_PATTERN = '(\s|$|٫|،)'
START_OF_PATTERN = '(\s|^|٫|،)'

PATTERN_UNIT_WITH_NUMERICAL_AMOUNT = f'{START_OF_PATTERN}(?P<amount>{PATTERN_SEARCH})(?P<unit>{PATTERN_PROD_UNIT}){ALL_YA}*{END_OF_PATTERN}'
PATTERN_UNIT_WITH_VALUE_OF_1 = join_patterns([ \
                f'{START_OF_PATTERN}?(?P<unit1>{PATTERN_PROD_UNIT}){ALL_YA}+({END_OF_PATTERN})' ,\
                f'{START_OF_PATTERN}?{HAR_PATTERN}{WHITE_SPACE}(?P<unit2>{PATTERN_PROD_UNIT})({END_OF_PATTERN})' ,\
                f'{START_OF_PATTERN}?(?P<unit3>{PATTERN_PROD_UNIT}){EI_PATTERN}({END_OF_PATTERN})'])
PATTERN_UNIT_SPECIAL_CASES = f"(?P<unit>{join_patterns(units['sp_units'])})"


PATTERN_PROD_UNIT_and_AMOUNT = join_patterns([f'(?P<amount>{PATTERN_SEARCH})(?P<unit0>{PATTERN_PROD_UNIT}){ALL_YA}*',\
                f'{START_OF_PATTERN}?(?P<unit1>{PATTERN_PROD_UNIT}){ALL_YA}+({END_OF_PATTERN})' ,\
                f'{START_OF_PATTERN}?(?P<unit2>{PATTERN_PROD_UNIT}){EI_PATTERN}({END_OF_PATTERN})'])
LEN_LONGEST_PROD_UNIT = 25
LEN_LONGEST_PATTERN_ABOUT_PROD = 30

In [ ]:

import json

    
class ProductUnitExtractor:
  def __init__(self):
    self.re_search_prod_unit_with_numerical_amount = re.compile(PATTERN_UNIT_WITH_NUMERICAL_AMOUNT)
    self.re_search_prod_unit_with_value_of_1 = re.compile(PATTERN_UNIT_WITH_VALUE_OF_1)
    self.re_search_prod_unit_special_cases = re.compile(PATTERN_UNIT_SPECIAL_CASES)
    # self.preprocessor = preprocessor

  def __extract_prod_unit_with_numerical_amount(self, input_sentence): #o
    matched_unit = None
    matched_span = None
    match = self.re_search_prod_unit_with_numerical_amount.search(input_sentence)
    if match != None:
      matched_unit = match["unit"]
      matched_span = match.span() 
      return True, matched_unit, matched_span
    
    return False, None, None

  def __extract_prod_unit_with_value_of_1(self, input_sentence): #o
    return_value = []
    for match in self.re_search_prod_unit_with_value_of_1.finditer(input_sentence):
      unit = [match["unit1"],match["unit2"],match["unit3"]]
      unit = [u for u in unit if u!=None]
      return_value.append(unit[0])
    if len(return_value) > 0:
      return True, return_value
    return False, []

  def __extract_prod_unit_special_cases(self, input_sentence): #o
    return_value = []
    for match in self.re_search_prod_unit_special_cases.finditer(input_sentence):
      return_value.append(match['unit'])
    if len(return_value) > 0:
      return True, return_value
    return False, []

  def extract_numbers(self, input_sentence):
    extractor = NumberExtractor()
    extracted_numbers = extractor.run(input_sentence)
    num_spans = []
    num_phrases = []
    for num_dict in extracted_numbers:
      span = num_dict['span']
      num_spans.append(span)

    return num_spans

  def run(self, input_sentence, number_spans):
    t_sentence =  input_sentence

    return_value = []    
    # input_sentence = self.preprocessor.preprocess(input_sentence)
    
    # number_spans = self.extract_numbers(input_sentence)
    matched_spans = []
    for num_span in number_spans:
      if num_span[1]+LEN_LONGEST_PROD_UNIT >= len(input_sentence):
        end = len(input_sentence) - 1
      
      else:
        end = num_span[1]+LEN_LONGEST_PROD_UNIT
      have_unit, matched_unit, matched_span = \
              self.__extract_prod_unit_with_numerical_amount(input_sentence[num_span[0] :end].lstrip())
      if have_unit:
        prod_amount = input_sentence[num_span[0]: num_span[1]]  
        return_value.append({
                    'product_amount': prod_amount,
                    'product_unit': matched_unit,
                })
        matched_len = matched_span[1] - matched_span[0]
        a_str = '-'.join(['' for _ in range(matched_len)])
        t_sentence = t_sentence[0:matched_span[0]+num_span[0]]+ a_str + t_sentence[matched_span[1]+num_span[0]:]
    
    have_unit, matched_units = self.__extract_prod_unit_with_value_of_1(t_sentence)
    if have_unit:
      for matched_unit in matched_units:
        return_value.append({
                    'product_amount': 'یک',
                    'product_unit': matched_unit,
                })
    have_unit, matched_units = self.__extract_prod_unit_special_cases(t_sentence)
    if have_unit:
      for matched_unit in matched_units:
        return_value.append({
                    'product_amount': 'یک',
                    'product_unit': matched_unit,
                })
         
    return return_value

# Part 4 - Final Model

<div dir=rtl>
این بخش تجمیع‌کننده‌ی سه بخش قبل می‌باشد که با ران کردن هر مدل، اطلاعات مربوطه را استخراج کرده و آن‌ها را تجمیع می‌کند و به عنوان نتیجه نهایی خروجی می‌دهد.

اما کار این مدل صرفا تجمیع سه مدل قبل نیست، بلکه برای بالا بردن دقت، ایده‌هایی در نظر گرفته شده است.

</div>

<div dir=rtl>

###  پیش پردازش 
</br>
پیش پردازشی که در این بخش انجام می دهیم شامل دو بخش می شود. در بخش اول با توجه به این که علامت های جداکننده در مدل number_extractor فقط  (. یا ٫ ) می باشند، سایر علامت های جداکننده مانند (/ یا ,) به یکی از این علامت های جداکننده معتبر تبدیل می شوند. در بخش دوم نیز فرمی از اعداد ۵ و ۶ به حالت معتبر آن تبدیل می شوند.


### استخراج قیمت
در این بخش ابتدا قیمت محصول و واحد آن استخراج می‌شود و در قالب یک لیست از دیکشنری‌ها برگردانده می‌شود.


### شکستن متن

حال برای سهولت کار، جلوگیری از خطاها در استخراج واحد و نام محصول، متن را بر اساس قیمت استخراج‌شده میشکنیم. به این صورت:

طبق اعلام ستاد موسوم به «تنظیم بازار کشور» قیمت هر کیلو مرغ ۵۹ هزار و ۸۰۰ تومان و هر کیلو تخم‌مرغ ۳۹ هزار و۸۰۰ تومان اعلام شد.

پس از استخراج قیمت‌ها، برای ادامه کار و درآوردن تعداد و واحد و اسم محصول، به این صورت متن را میشکنیم:

طبق اعلام ستاد موسوم به «تنظیم بازار کشور» قیمت هر کیلو مرغ

 و هر کیلو تخم‌مرغ
 
 دقت کنید که قیمت را از آن حذف نموده‌ایم.
 
 این کار در بالا بردن دقت تاثیر بسزایی دارد. چرا که کلمات کمتری نیاز به بررسی دارند. مثلا اگر در جمله ای هم سکه و هم دلار بیان شده بود، هر دو پتانسیل نام محصول را دارند. ولی اگر دلار به عنوان واحد قیمت باشد و در اینجا حذف شود به راحتی میتوان سکه را به عنوان محصول در نظر گرفت.
 
### استخراج اندازه و واحد
 
 در این بخش به ازای هر بخش شکسته‌شده، اندازه و واحد آن را به دست می‌آوریم.
 
### جایگذاری اندازه و واحد
 
 در مرحله بعد، اندازه و واحد آن استخراج میشود و به ازای آن علامت "-" میگذاریم.
 
 یعنی بخش دوم که "و هر کیلو تخم‌مرغ" بود تبدیل می‌شود به

--------تخم‌مرغ

که در این صورت به راحتی نام محصول به دست می‌آید.

در واقع در اینجا ما با کم کردن گزینه‌های موجود، دقت مدل را بالا برده‌ایم و از طرفی تمهیداتی در نظر گرفتیم تا span ها درست استخراج شوند.

### حذف به قرینه معنوی

یکی دیگر از ایده‌های دیگری که در نظر گرفته شده این است که در متونی که اسم محصول یک‌بار بیان شده و دفعه بعد به قرینه معنوی حذف شده، اسم قبلی را در نظر می‌گیریم. برای مثال:

قیمت مصوب مرغ به حدود ۵۰هزار تومان رسید، اما به دلیل مازاد عرضه، با قیمتی در حدود ۳۶ تا ۴۰ هزار تومان به دست مصرف‌کننده رسید.

در اینجا برای بخش دوم هم نام محصول را "مرغ" در نظر می‌گیریم.

### متن‌های بدون واحد

برای متونی که واحد و اندازه ندارند، اگر سرویس باشند واحدشان "سرویس" و مقدارشان "یک" در نظر گرفته می‌شود. همچنین اگر محصول باشند واحدشان "عدد" و مقدارشان "یک" در نظر گرفته می‌شود.

</div>


In [ ]:
from typing import Iterable, Tuple, List

class Model:
  def __init__(self):
    self._product_name_extractor = ProductNameExtractor()
    self._product_price_extractor = ProductPriceExtractor()
    self._product_unit_extractor = ProductUnitExtractor()
    self._preprocessor = Preprocessor()
    self._last_valid_product_name = None
    self._last_valid_product_span = None


  def extract_numbers(self, input_sentence):
    extractor = NumberExtractor()
    extracted_numbers = extractor.run(input_sentence)
    num_spans = []
    num_phrases = []
    for num_dict in extracted_numbers:
      span = num_dict['span']
      num_spans.append(span)
    return num_spans

  def run(self, input_text: str):
    self._last_valid_product_name = None
    self._last_valid_product_span = None

    input_text = self._preprocessor.preprocess(input_text)
    number_spans = self.extract_numbers(input_text)
    final_results = self._product_price_extractor.run(input_text, number_spans)
    remaining_text = input_text

    previouse_sections_len = 0

    for index, result_dict in enumerate(final_results):
      current_section, remaining_text, remaining_start_index = self._split_into_different_product_sentences(
            remaining_text, result_dict['price_amount'], result_dict['price_unit']
      )
      self._extract_name_and_unit(
          current_section, result_dict,
          previouse_sections_len, number_spans
      )

      previouse_sections_len += remaining_start_index

    return final_results

  def _extract_name_and_unit(
      self, current_section, result_dict,
      previouse_sections_len,
      number_spans
  ):
    product_amount, product_unit = self._extract_product_unit(result_dict, current_section, number_spans)
    current_section = current_section.replace(product_amount, '-'*len(product_amount))
    current_section = current_section.replace(product_unit, '-'*len(product_unit))

    product_name, product_span = self._extract_product_name(current_section)

    if not product_amount:
      self._fill_empty_amount_and_unit(product_name, result_dict)

    if product_name is not None and product_span is not None:
      product_span[0] += previouse_sections_len
      product_span[1] += previouse_sections_len
      product_name, product_span = self._strip_names_and_update_spans(product_name, product_span)
      self._last_valid_product_name = product_name
      self._last_valid_product_span = product_span

    else:
      product_name = self._last_valid_product_name
      product_span = self._last_valid_product_span

    result_dict['product_name'] = product_name
    result_dict['product_name_span'] = product_span

  def _fill_empty_amount_and_unit(self, product_name, result_dict):
    if product_name is None:
      return
    if self._product_name_extractor.is_service(product_name):
      result_dict['product_amount'] = 'یک'
      result_dict['product_unit'] = 'سرویس'
    else:
      result_dict['product_amount'] = 'یک'
      result_dict['product_unit'] = 'عدد'

  def _extract_product_unit(self, result_dict, text, number_spans):
      product_unit_result = self._product_unit_extractor.run(text, number_spans)

      if product_unit_result:
        product_amount = product_unit_result[-1]['product_amount']
        product_unit = product_unit_result[-1]['product_unit']
      else:
        product_amount = ''
        product_unit = ''

      result_dict['product_amount'] = product_amount
      result_dict['product_unit'] = product_unit

      return product_amount, product_unit

  def _extract_product_name(self, text):
    product_name_result = self._product_name_extractor.run(text)

    product_name = product_name_result['product_name']
    product_span = product_name_result['product_name_span']

    return product_name, product_span

  def _strip_names_and_update_spans(self, name: str, span: List[int]):
    span[0] += (len(name) - len(name.lstrip()))
    name = name.lstrip()
    span[1] -= (len(name) - len(name.rstrip()))
    name = name.rstrip()

    return name, span

  def _split_into_different_product_sentences(self, text: str, price_amount: str, price_unit: str):
    partitions = text.partition(price_amount)
    current_part = partitions[0]

    remainig_start_index = len(current_part) + len(partitions[1])

    remaining_part = partitions[-1]

    if price_unit:
      units_partitions = remaining_part.partition(price_unit)
      remaining_part = units_partitions[-1]
      remainig_start_index += len(units_partitions[0]) + len(units_partitions[1])
    return current_part, remaining_part, remainig_start_index


In [ ]:
def print_with_index(text):
  for index, char in enumerate(text):
    print(f'{char}-- {index}', end=' : ')

# Final Result

In [ ]:
from typing import Dict, List

def score(results: List[Dict], true_output: List[Dict]):
  all_fields = 0
  true_fields = 0

  for index, result_dict in enumerate(results):
    for key, value in result_dict.items():
      all_fields += 1
      if index >= len(true_output):
        continue
      if value == true_output[index][key]:
        true_fields += 1


  return true_fields, all_fields

In [ ]:
true_outputs = [
    [{'price_amount': 'یک میلیون', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیسه', 'product_name': 'برنج', 'product_name_span': [0, 4]}],
 [{'price_amount': '۲۰۰ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'بطری', 'product_name': 'روغن', 'product_name_span': [5, 9]}],
 [{'price_amount': 'یکصد هزار', 'price_unit': 'تومان', 'product_amount': 'نیم', 'product_unit': 'کیلو', 'product_name': 'روغن', 'product_name_span': [0, 4]}],
 [{'price_amount': '۱۵ هزار', 'price_unit': 'تومان', 'product_amount': '700', 'product_unit': 'گرم', 'product_name': 'ماکارانی', 'product_name_span': [0, 8]}],
 [{'price_amount': '۸۰۰.۸', 'price_unit': 'تومان', 'product_amount': '60', 'product_unit': 'عدد', 'product_name': 'نان', 'product_name_span': [0, 3]}],
 [{'price_amount': '۵۹ هزار و ۸۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'مرغ', 'product_name_span': [56, 59]}, {'price_amount': '۳۹ هزار و۸۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'تخم مرغ', 'product_name_span': [90, 97]}],
 [{'price_amount': '۱۲۰٫۰۰۰', 'price_unit': 'ریال', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'شیر', 'product_name_span': [27, 30]}],
 [{'price_amount': '۱۵۰٫۰۰۰', 'price_unit': 'ریال', 'product_amount': '۹۰۰', 'product_unit': 'گرم', 'product_name': 'شیر', 'product_name_span': [16, 19]}],
 [{'price_amount': '۱۸۰٫۰۰۰', 'price_unit': 'ریال', 'product_amount': 'یک', 'product_unit': 'لیتر', 'product_name': 'شیر', 'product_name_span': [16, 19]}],
 [{'price_amount': '۴۹۸٫۰۰۰', 'price_unit': 'ریال', 'product_amount': '۲.۵', 'product_unit': 'کیلو', 'product_name': 'ماست', 'product_name_span': [16, 20]}],
 [{'price_amount': '۳۷۰٫۰۰۰', 'price_unit': 'ریال', 'product_amount': '۴۰۰', 'product_unit': 'گرم', 'product_name': 'پنیر', 'product_name_span': [16, 20]}],
 [{'price_amount': '۶۳۰٫۰۰۰', 'price_unit': 'ریال', 'product_amount': '۸۱۰', 'product_unit': 'گرم', 'product_name': 'روغن', 'product_name_span': [16, 20]}],
 [{'price_amount': '۶۲۰٫۰۰۰', 'price_unit': 'ریال', 'product_amount': '۸۱۰', 'product_unit': 'گرم', 'product_name': 'روغن', 'product_name_span': [16, 20]}],
 [{'price_amount': '۴۰۰۰  به ۱۲ هزار و ۳۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'ذرت', 'product_name_span': [5, 8]}],
 [{'price_amount': '۵۹۸٫۰۰۰', 'price_unit': 'ریال', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'مرغ', 'product_name_span': [27, 30]}],
 [{'price_amount': '۳۹۸٫۰۰۰', 'price_unit': 'ریال', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'تخم مرغ', 'product_name_span': [27, 34]}],
 [{'price_amount': '۴۲۲ میلیون', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'ریو', 'product_name_span': [4, 7]}],
 [{'price_amount': '77 هزار', 'price_unit': 'دلار', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'سورنتو', 'product_name_span': [9, 15]}],
 [{'price_amount': '19 هزار و 400', 'price_unit': 'دلار', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'سراتو', 'product_name_span': [15, 20]}],
 [{'price_amount': '۲۶ تا ۳۰ هزار', 'price_unit': 'دلار', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'سانتافه', 'product_name_span': [11, 18]}, {'price_amount': '۳۰ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'دلار', 'product_name_span': [57, 61]}, {'price_amount': '۹۰۰ میلیون', 'price_unit': 'تومان', 'product_amount': '', 'product_unit': '', 'product_name': 'سانتافه', 'product_name_span': [11, 18]}],
 [{'price_amount': '۵۰۰ میلیون', 'price_unit': '', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'مگان', 'product_name_span': [0, 4]}],
 [{'price_amount': '۳۶۴٫۷۲۰', 'price_unit': 'ریال', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'یورو', 'product_name_span': [4, 8]}, {'price_amount': '۴۱۹٫۷۸۰', 'price_unit': 'ریال', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'پوند', 'product_name_span': [42, 46]}],
 [{'price_amount': '۲۰', 'price_unit': 'دلار', 'product_amount': 'یک', 'product_unit': 'نفر', 'product_name': 'هتل', 'product_name_span': [26, 29]}],
 [{'price_amount': '۲۵۰', 'price_unit': 'دینار', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'نان', 'product_name_span': [112, 115]}, {'price_amount': '۲۵۰', 'price_unit': 'دینار', 'product_amount': 'یک', 'product_unit': 'استکان', 'product_name': 'چای', 'product_name_span': [139, 142]}],
 [{'price_amount': '2', 'price_unit': 'درهم', 'product_amount': 'یک', 'product_unit': 'کیلومتر', 'product_name': 'تاکسی', 'product_name_span': [5, 10]}],
 [{'price_amount': '125', 'price_unit': 'درهم', 'product_amount': 'نیم', 'product_unit': 'ساعت', 'product_name': 'جت اسکی', 'product_name_span': [6, 13]}],
 [{'price_amount': '۱۰۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'صفحه', 'product_name': 'چاپ', 'product_name_span': [0, 3]}],
 [{'price_amount': '۳۸۰۰۰', 'price_unit': 'ریال', 'product_amount': 'یک', 'product_unit': 'صفحه', 'product_name': 'تحقیق', 'product_name_span': [0, 5]}],
 [{'price_amount': 'شش میلیون', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'ماهیانه', 'product_name': 'حقوق', 'product_name_span': [47, 51]}],
 [{'price_amount': '۱۰۰ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'ساعت', 'product_name': 'نظافت', 'product_name_span': [0, 5]}],
 [{'price_amount': '۳۵ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'وتیروکسین', 'product_name_span': [0, 9]}],
 [{'price_amount': '۵۱ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'ورق', 'product_name': 'اورسودزوکسی', 'product_name_span': [11, 22]}],
 [{'price_amount': '۱٫۴۹۴٫۴۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'گرم', 'product_name': 'طلا ۱۸ عیار', 'product_name_span': [7, 18]}],
 [{'price_amount': '۶٫۴۷۳٫۰۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'مثقال', 'product_name': 'طلا', 'product_name_span': [9, 12]}],
 [{'price_amount': '۱٫۷۴۵', 'price_unit': '', 'product_amount': 'یک', 'product_unit': 'اونس', 'product_name': 'طلا', 'product_name_span': [8, 11]}],
 [{'price_amount': '۱۱۳۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'جو و ذرت', 'product_name_span': [28, 36]}, {'price_amount': '۱۶۳۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'سویا', 'product_name_span': [58, 62]}],
 [{'price_amount': '۲۵۷', 'price_unit': 'یورو ', 'product_amount': 'یک', 'product_unit': 'تن', 'product_name': 'جو', 'product_name_span': [47, 49]}],
 [{'price_amount': '۵۰٫۰۰۰', 'price_unit': 'ریال', 'product_amount': 'یک', 'product_unit': 'شاخه', 'product_name': 'نرگس', 'product_name_span': [38, 42]}],
 [{'price_amount': '۲۰ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'بسته های فرهنگی', 'product_name_span': [0, 15]}],
 [{'price_amount': '۲ میلیارد', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'ماشین', 'product_name_span': [0, 5]}],
 [{'price_amount': '۱۰۰ میلیون', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'دستگاه', 'product_name': 'تلویزیون', 'product_name_span': [0, 8]}],
 [{'price_amount': '۱٫۲۵۰٫۰۰۰', 'price_unit': 'تومان', 'product_amount': '۱۰', 'product_unit': 'کیلو', 'product_name': 'پمپ تخلیه', 'product_name_span': [0, 9]}],
 [{'price_amount': '۱۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلومتر', 'product_name': 'باربری', 'product_name_span': [6, 12]}],
 [{'price_amount': '۱۳ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'گندم', 'product_name_span': [52, 56]}],
 [{'price_amount': '۳۳۱', 'price_unit': 'یورو ', 'product_amount': 'یک', 'product_unit': 'تن', 'product_name': 'گندم', 'product_name_span': [18, 22]}],
 [{'price_amount': '۱۰۱۶۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'گندم', 'product_name_span': [11, 15]}],
 [{'price_amount': '۵۰۰ میلیارد', 'price_unit': 'دلار', 'product_amount': '۲ میلیارد', 'product_unit': 'تن', 'product_name': 'زباله', 'product_name_span': [26, 31]}],
 [{'price_amount': 'یک میلیون و ۳۵۹ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'گرم', 'product_name': 'طلای۱۸ عیار', 'product_name_span': [75, 86]}, {'price_amount': '۱۵ میلیون و ۱۰۰ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'سکه تمام بهار آزادی', 'product_name_span': [128, 147]}],
 [{'price_amount': '۵۳۰', 'price_unit': 'دلار', 'product_amount': 'یک', 'product_unit': 'تن', 'product_name': 'شکر', 'product_name_span': [25, 28]}],
 [{'price_amount': '۲۴ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'سرویس', 'product_name': 'نظافت', 'product_name_span': [31, 36]}],
 [{'price_amount': '۶۰.۱ میلیارد', 'price_unit': 'دلار', 'product_amount': '۸۱ میلیون', 'product_unit': 'تن', 'product_name': 'مبادلات تجاری', 'product_name_span': [0, 13]}],
 [{'price_amount': '۱۶ تا ۲۲ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'پیاز', 'product_name_span': [85, 89]}, {'price_amount': '۱۶ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'گوجه', 'product_name_span': [111, 115]}, {'price_amount': '۱۷ هزار و ۵۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'خیار', 'product_name_span': [142, 146]}, {'price_amount': '۱۵ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'سیب زمینی', 'product_name_span': [168, 171]}, {'price_amount': '۴۳ تا ۵۳ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'موز', 'product_name_span': [193, 196]}, {'price_amount': '۴۷ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'سیب', 'product_name_span': [218, 221]}, {'price_amount': '۱۸ هزار و ۹۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'پرتقال', 'product_name_span': [242, 248]}, {'price_amount': '۱۷ تا ۳۳ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'نارنگی', 'product_name_span': [270, 276]}, {'price_amount': '۳۱ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'گلابی', 'product_name_span': [299, 304]}],
 [{'price_amount': '59 هزار و 500', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'مرغ', 'product_name_span': [5, 8]}, {'price_amount': '63 هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'مرغ', 'product_name_span': [5, 8]}, {'price_amount': '52 هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'مرغ', 'product_name_span': [153, 156]}],
 [{'price_amount': 'یک میلیارد و ۱۹۵ میلیون', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'کی ام سی', 'product_name_span': [24, 32]}],
 [{'price_amount': '۱۵', 'price_unit': 'ریال', 'product_amount': 'یک', 'product_unit': 'بطری', 'product_name': 'کوکاکولا', 'product_name_span': [28, 36]}, {'price_amount': '۳.۶', 'price_unit': 'ریال', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'دلار', 'product_name_span': [147, 151]}],
 [{'price_amount': '۹ هزار و ۷۰۰  تا ۱۲ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'بطری', 'product_name': 'نوشابه', 'product_name_span': [5, 11]}],
 [{'price_amount': '۲۹۴ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'پیاله', 'product_name': 'بادام زمینی', 'product_name_span': [19, 30]}, {'price_amount': '۳۴۳ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'پیاله', 'product_name': 'پسته', 'product_name_span': [83, 87]}],
#57 ...
[{'price_amount': '۱۵۰ میلیون', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'متر', 'product_name': 'زمین', 'product_name_span': [27, 31]},{'price_amount': '۳۰ میلیون','price_unit': 'تومان','product_amount': 'یک','product_unit': 'متر','product_name': 'زمین','product_name_span': [27, 31]}],
[{'price_amount': '60','price_unit': 'دلار','product_amount': 'یک','product_unit': 'بشکه','product_name': 'نفت','product_name_span': [54, 57]},{'price_amount': '40 تا 50','price_unit': 'دلار','product_amount': 'یک','product_unit': 'بشکه','product_name': 'نفت','product_name_span': [175, 178]}],
[{'price_amount': '229 میلیون','price_unit': 'تومان','product_amount': 'یک','product_unit': 'عدد','product_name': 'کوییکR','product_name_span': [96, 102]},{'price_amount': '222 میلیون','price_unit': 'تومان','product_amount': 'یک','product_unit': 'عدد','product_name': 'کوییک معمولی','product_name_span': [149, 161]}],
[{'price_amount': '۳۳هزار و ۵۰۰','price_unit': 'تومان','product_amount': 'یک','product_unit': 'عدد','product_name': 'دلار','product_name_span': [15, 19]}],
[{'price_amount': '۱۵ هزار','price_unit': 'تومان','product_amount': 'یک','product_unit': 'کیلوگرم','product_name': 'پیاز قرمز','product_name_span': [25, 34]},{'price_amount': '۱۲ هزار','price_unit': 'تومان','product_amount': 'یک','product_unit': 'کیلوگرم','product_name': 'پیاز شیری','product_name_span': [56, 65]},{'price_amount': '۱۴ هزار','price_unit': 'تومان','product_amount': 'یک','product_unit': 'کیلوگرم','product_name': 'پیاز سفید','product_name_span': [81, 90]},{'price_amount': '۱۲ هزار','price_unit': 'تومان','product_amount': 'یک','product_unit': 'کیلوگرم','product_name': 'پیاز زرد','product_name_span': [106, 114]},{'price_amount': '۱۰ تا ۱۳ هزار','price_unit': 'تومان','product_amount': 'یک','product_unit': 'کیلوگرم','product_name': 'سیب زمینی','product_name_span': [130, 139]},{'price_amount': '۵ تا ۱۰ هزار','price_unit': 'تومان','product_amount': 'یک','product_unit': 'کیلوگرم','product_name': 'گوجه فرنگی','product_name_span': [162, 172]}],
[{'price_amount': '۱۶ هزار و ۶۵۰','price_unit': 'دلار','product_amount': 'یک','product_unit': 'عدد','product_name': 'بیت‌کوین','product_name_span': [96, 102]},{'price_amount': 'هزار و ۲۰۱','price_unit': 'دلار','product_amount': 'یک','product_unit': 'عدد','product_name': 'اتریوم','product_name_span': [96, 102]},],
[{'price_amount': '۲ میلیون', 'price_unit': '', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'ساینا', 'product_name_span': [0, 5]},{'price_amount': '۲۲۵ میلیون', 'price_unit': 'تومان','product_amount': 'یک','product_unit': 'عدد','product_name': 'ساینا','product_name_span': [41, 46]}],
[{'price_amount': '۲ میلیون و ۲۹ هزار و ۹۴۰','price_unit': 'تومان','product_amount': 'یک','product_unit': 'گرم','product_name': 'طلای ۲۴ عیار','product_name_span': [7, 19]}],
[{'price_amount': '۱۶میلیون و ۶۰۴ هزار','price_unit': 'تومان','product_amount': 'یک','product_unit': 'عدد','product_name': 'سکه','product_name_span': [4, 7]}],
[{'price_amount': '۸۰ هزار','price_unit': 'تومان','product_amount': 'یک','product_unit': 'سرویس','product_name': 'کوتاهی مو','product_name_span': [6, 15]}],
[{'price_amount': '۱۰۰ تا ۲۰۰ ‌هزار','price_unit': 'تومان','product_amount': 'یک','product_unit': 'سرویس','product_name': 'کاشت مژه','product_name_span': [7, 15]},{'price_amount': '۱۰۰ هزار','price_unit': 'تومان','product_amount': 'یک','product_unit': 'سرویس','product_name': 'کاشت مژه','product_name_span': [7, 15]}],
[{'price_amount': '۲۰ هزار','price_unit': 'تومان','product_amount': 'یک','product_unit': 'سرویس','product_name': 'زیرشویی','product_name_span': [4, 11]},{'price_amount': '۳۰ هزار','price_unit': 'تومان','product_amount': 'یک','product_unit': 'سرویس','product_name': 'زیرشویی','product_name_span': [4, 11]},{'price_amount': '۲۲ هزار','price_unit': 'تومان','product_amount': 'یک','product_unit': 'سرویس','product_name': 'زیرشویی','product_name_span': [4, 11]}],
[{'price_amount': '۹ هزار و ۲۴۲', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'درهم', 'product_name_span': [25, 29]}, {'price_amount': '۹ هزار و ۳۹۷', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'درهم', 'product_name_span': [25, 29]}],
[{'price_amount': 'هزار و ۹۱۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'لیر', 'product_name_span': [5, 8]}],
[{'price_amount': '300٫000', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'سرویس', 'product_name': 'باربری', 'product_name_span': [28, 34]}],
[{'price_amount': '۱۰ تا ۴۳ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'انار', 'product_name_span': [97, 101]}, {'price_amount': '۲۰ تا ۴۰ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'انگور', 'product_name_span': [135, 140]}],
[{'price_amount': '۸ تا ۱۸ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'خرمالو', 'product_name_span': [41, 47]}, {'price_amount': '۱۵ تا ۳۷ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'خرمالو', 'product_name_span': [41, 47]}],
[{'price_amount': '40٫000', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'سرویس', 'product_name': 'اصلاح سر', 'product_name_span': [10, 18]}],
[{'price_amount': '100 هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'سرویس', 'product_name': 'اصلاح سر', 'product_name_span': [21, 29]}],
[{'price_amount': '۱۱۸۰', 'price_unit': 'دلار', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'اتریوم', 'product_name_span': [18, 24]}],
[{'price_amount': '۱۰ میلیارد', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'قواره', 'product_name': 'زمین', 'product_name_span': [19, 23]}],
[{'price_amount': '۲۷ تا ۵۶ میلیون', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'یخچال', 'product_name_span': [50, 55]}],
[{'price_amount': '۳۳۱ میلیون', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'پژو ۲۰۶', 'product_name_span': [0, 7]}],
[{'price_amount': '۱۶۳۸۸.۴۰', 'price_unit': 'دلار', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'بیت کوین', 'product_name_span': [0, 8]}],
[{'price_amount': '۹۰۰ میلیون', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'آپارتمان', 'product_name_span': [31, 39]}],
[{'price_amount': '۶۹۵ میلیون', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'عدد', 'product_name': 'تندر ۹۰', 'product_name_span': [5, 12]}],
[{'price_amount': '۵۰هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'مرغ', 'product_name_span': [10, 13]}, {'price_amount': '۳۶ تا ۴۰ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلوگرم', 'product_name': 'مرغ', 'product_name_span': [10, 13]}]
]

In [ ]:
print(len(true_outputs))

84


In [ ]:
from dataclasses import dataclass

@dataclass
class EstimationData:
  index: int
  sample: str
  results: list
  true_output: list

In [ ]:
model = Model()

all_extractions = 0
true_extractions = 0

false_estimations = []

for index, sample in enumerate(samples):
  print(f'sample {index}: ', sample)
  results = model.run(sample)
  print('Result: ', results)
  # print_with_index(sample)
  print('\n\n')

  true_fields, all_fields = score(results, true_outputs[index])

  if true_fields != all_fields:
    false_estimations.append(EstimationData(index, sample, results, true_outputs[index]))

  all_extractions += all_fields
  true_extractions += true_fields

print('true_extractions:', true_extractions)
print('all_extractions:', all_extractions)

print('score:', true_extractions/all_extractions)

sample 0:  برنج سر لاشه گلچین امساله اعلاء ۱۰ کیلویی (تضمین کیفیت) هر کیسه با قیمت یک میلیون تومان به بازار عرضه می شود.
Result:  [{'price_amount': 'یک میلیون', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیسه', 'product_name': 'برنج', 'product_name_span': [0, 4]}]



sample 1:  قیمت روغن مخصوص سرخ کردنی بدون پالم اویلا ۱.۸ کیلوگرمی ، هر بطری ۲۰۰ هزار تومان می باشد.
Result:  [{'price_amount': '۲۰۰ هزار', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'بطری', 'product_name': 'روغن', 'product_name_span': [5, 9]}]



sample 2:  روغن حیوانی گاوی اعلاء نیم کیلویی با قیمت یکصد هزار تومان به دست مشتری می رسد.
Result:  [{'price_amount': 'یکصد هزار', 'price_unit': 'تومان', 'product_amount': 'نیم', 'product_unit': 'کیلو', 'product_name': 'روغن', 'product_name_span': [0, 4]}]



sample 3:  ماکارانی زر ماکارون قطر 1.5 وزن 700 گرم با قیمت ۱۵ هزار تومان برای هر بسته در فروشگاه های کشور توزیع می شود.
Result:  [{'price_amount': '۱۵ هزار', 'price_unit': 'تومان', 'pro

### Print False Estimations

In [ ]:
for estimation_data in false_estimations:
  print(f'sample {estimation_data.index}: {estimation_data.sample}')
  print(f'result: {estimation_data.results}')
  print(f'true: {estimation_data.true_output}')

sample 5: طبق اعلام ستاد موسوم به «تنظیم بازار کشور» قیمت هر کیلو مرغ ۵۹ هزار و ۸۰۰ تومان و هر کیلو تخم‌مرغ ۳۹ هزار و۸۰۰ تومان اعلام شد
result: [{'price_amount': '۵۹ هزار و ۸۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'مرغ', 'product_name_span': [56, 59]}, {'price_amount': '۳۹ هزار و۸۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'مرغ', 'product_name_span': [56, 59]}]
true: [{'price_amount': '۵۹ هزار و ۸۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'مرغ', 'product_name_span': [56, 59]}, {'price_amount': '۳۹ هزار و۸۰۰', 'price_unit': 'تومان', 'product_amount': 'یک', 'product_unit': 'کیلو', 'product_name': 'تخم مرغ', 'product_name_span': [90, 97]}]
sample 19: قیمت جهانی سانتافه ۲۰۲۲ بین ۲۶ تا ۳۰ هزار دلار است که با دلار ۳۰ هزار تومانی در داخل کشور یعنی ۹۰۰ میلیون تومان می‌شود
result: [{'price_amount': '۲۶ تا ۳۰ هزار', 'price_unit': 'دلار', 'product_am